In [ ]:
pip uninstall -y yfinance

#pip install curl_cffi
#import curl_cffi
# Remplacer vos sessions requests par curl_cffi
#session = curl_cffi.Session(impersonate="chrome", timeout=15)
# Puis utiliser cette session avec yfinance
#data = yf.download(ticker, session=session, ...)

In [1]:
pip install --upgrade yfinance --no-cache-dir



Note: you may need to restart the kernel to use updated packages.


In [ ]:
import yfinance as yf
import pandas as pd
import os
import json
import time
import logging
import random
from datetime import datetime, timedelta
from tqdm import tqdm
from typing import Dict, List, Optional, Tuple
import numpy as np
from pathlib import Path
import concurrent.futures


PARIS_TICKERS = [
#CAC40
    "MC.PA","OR.PA","SU.PA","AIR.PA","TTE.PA","SAN.PA","CDI.PA","EL.PA","SAF.PA","AI.PA","BNP.PA","CS.PA","AXA SA","DG.PA","DSY.PA","SGO.PA","BN.PA","ACA.PA","ENGI.PA","KER.PA","HO.PA","CAP.PA","RI.PA","LR.PA","ORA.PA","PUB.PA","GLE.PA",
    "ML.PA","DIM.PA","VIE.PA","AM.PA","BOL.PA","RNO.PA","BVI.PA","AMUN.PA","BIM.PA","AC.PA","EN.PA","ENX.PA","ADP.PA","URW.PA","SW.PA","IPN.PA","RNL.PA","ERF.PA","ALO.PA","CA.PA","FGR.PA","LI.PA","GET.PA","EDEN.PA","RXL.PA","IAM.PA","CBDG.PA","GFC.PA",
    "FDJ.PA","ODET.PA","COTY.PA","AKE.PA","AYV.PA","RF.PA","COV.PA","GTT.PA","SPIE.PA","SPIE SA","TEP.PA","SK.PA","SEB SA","TE.PA","ELIS.PA","Elis SA","SCR.PA","VK.PA","NEX.PA","MF.PA","MLHK.PA","TKO.PA","FLY.PA","SOP.PA",
    "DEC.PA","PLX.PA","ITP.PA","VRLA.PA","SOI.PA","RCO.PA","COVH.PA","MMB.PA","ATE.PA","VU.PA","FR.PA","IDL.PA","BB.PA","RUI.PA","VIRP.PA","TRI.PA","BAIN.PA","VIV.PA","COFA.PA","CARM.PA","LOUP.PA","NK.PA","WLN.PA","ALTA.PA","UNBL.PA","IPS.PA",
    "CAF.PA","AF.PA","PLNW.PA","CBE.PA","VCT.PA","PEUG.PA","RBT.PA","EXN.PA","STF.PA","STEF SA","ICAD.PA","SESG.PA","OPM.PA","ERA.PA","ARG.PA","TFI.PA","TF1 SA","UBI.PA","OVH.PA","MMT.PA","ES.PA","BLV.PA","MAU.PA","GDS.PA","FII.PA","WAVE.PA",
    "CRLA.PA","NRO.PA","LSS.PA","MERY.PA","ETL.PA","ELEC.PA","FREY.PA","DBG.PA","CNDF.PA","LTA.PA","CDA.PA","FNAC.PA","MTU.PA","VETO.PA","TKTT.PA","VIL.PA","BEN.PA","EC.PA","VAC.PA","SAVE.PA","BASS.PA","NXI.PA","XFAB.PA","SDG.PA",
    "THEP.PA","CRAV.PA","CRSU.PA","CRAP.PA","CEN.PA","SCHP.PA","TFF.PA","KOF.PA","QDT.PA","LPE.PA","SBT.PA","EQS.PA","BUR.PA","AUB.PA","GLO.PA",
]

US_ETF = ["SOXL","SPXS","FXI","TSLL","TSLZ","SQQQ","TQQQ","SOXS","ETHU","NVDQ","SPY","KWEB","IBIT","MSTU","XLF","EEM","TLT","HYG","UVXY","EWZ","TZA","QQQ","IWM","SLV","FAZ","LQD","NVDL","LABD","AMDL","SDS","MSTZ","GDX","NVDX","SCHD","VEA","EFA","TNA","SPXU",
          "YINN","SCHF","XBI","KRE","BITX","XLE","XLV","BITO","USHY","IEMG","SCHX","XLI","UVIX","ARKK","ASHR","IEFA","EMXC","GLD","RWM","VOO","VWO","MCHI","IJH","ETHA","NVDD","XLP","MSTX","JAAA","SCHG","EWJ","XLU","SMH","IAU","TSLS","SGOL","QID","QYLD","XLB",
          "XRT","NVD","SGOV","RSP","TMF","KORU","TSLQ","BND","BIL","AGG","BKLN","SCHH","SPLG","UNG","EMLC","INDA","FEZ","CONY","SCHB","SPTI","YANG","VCIT","FNGD","GOVT","IGV","MSOS","SPDN","SRTY","GDXJ","SBIT","IQLT","VXX","SHV","PSQ","EMB","VXUS","DRIP","JEPQ",
          "IYR","VTEB","SPIB","IEF","MSTY","JPST","UPRO","FBTC","XLK","SPDW","EWH","JEPI","ILF","PGX","CONL","IJR","IYZ","SPHY","CGGR","GGLL","EWY","KOLD","SCHP","VTI","CGDV","SHY","ITB","SILJ","IVV","SPTL","BITU","SOXX","SRLN","SPXL","SDOW","XHB","TIP","EZA",
          "ETHE","XLC","FBND","VGIT","USFR","ULTY","VCRB","HEFA","GBTC","ARKG","TSLY","VGSH","XLY","SJNK","ACWI","ICLN","VTV","AMZU","MAGS","AGQ","GLDM","XOP","JCPB","VNQ","SCHA","PFF","PAVE","PDBC","EFV","QQQM","BNDX","DOG","BOIL","EWT","IUSB","EZU","BITB",
          "SPYV","JNK","SSO","BSCT","BABX","DIA","QLD","IBB","XME","RSHO","EWW","SPSM","SCHO","EWC","VEU","CALF","SCHM","SPYG","USO","BITI","TSLT","VGK","MUB","SPMD","COWZ","ETH","TMV","COPX","UDOW","AMLP","URA","NVDY","KBWB","PULS","EWA","BSV","IDEV","EWG",
          "CLOZ","VMBS","SPTS","IXUS","SVIX","VIXY","DFAC","SDVY","TFLO","SCHZ","MLPX","SPEM","SPSB","LABU","DYNF","IGSB","CGUS","ETHT","BLV","BBJP","UCO","YMAX","IAUM","VCSH","AAAU","VGLT","VTWO","SPMO","TSLR","DGRO","SPAB","KBE","AUSF","PYLD","VCLT","NVDU",
          "JETS","IWD","CWEB","FNDF","SCHR","DXD","FLOT","IGIB","BIV","JBBB","MOAT","DFIC","MBB","URNM","TBIL","QEFA","SCHE","PVAL","FGD","IEUR","XLG","NUGT","IVW","BUFR","DFSV","OUNZ","SCHI","VFLO","CQQQ","DFAI","FETH","SPLV","NVDS","GRNY","MSFU","AAPU","IEI",
          "EEMA","CIBR","CGCP","ITOT","SHYG","BSCR","IWF","TECL","RDVY","QGRW","FPE","PXH","BAR","TFI","IWP","BINC","MTUM","UUP","ACWX","PZA","GDXD","MINT","AAPD","YMAG","SPLB","IWR","USMV","TECS","VUG","ACES","QUAL","IGLB","SVXY","SCHV","BTC","IYT","AMDY","FAS",
          "AVGX","SPYD","MRNY","ARKB","VIG","ZSL","URTY","NAIL","DFLV","TCAF","ITA","KIE","URNJ","HYD","TSDD","SPYI","VUSB","VTIP","FDVV","HTRB","SCO","FTSM","AMZZ","FLTR","BIZD","PTIR","DIVO","COWG","VSS","FBL","DFEM","EPI","IOO","AVUV","VONG","VYM","FNDE","SPHQ",
          "FLRN","HYEM","MDY","SIVR","USIG","DUST","DIVI","BUG","IWB","GSLC","UCON","CHAU","AVDV","BOTZ","JDST","FNDA","IBDQ","GUNR","PAAA","AIQ","VWOB","SIL","JNUG","DBA","SCZ","DUHP","SVOL","AVEM","CGMS","DFSD","HYLB","TWM","FENY","DFCF","GDXU","XSMO","AMZY","REM"
          ,"DRN","SLYV","IYW","BSCP","BSCQ","AIYY","CGBL","DBEF","FELC","GLL","EWU","FNGU","WEAT","DFAE","USD","CGGO","EDZ","GBIL","IUSV","EFG","BTCZ","IBDV","SPTM","SPYU","DPST","OIH","EDV","AMZD","DJAN","FALN","EUFN","PFFD","METU","AIRR","ETHD","TLTW","DFAU",
          "CETH","VFH","FBCG","IDV","BSCS","GSY","ICSH","ESGE","JGRO","DBC","CTA","FIAT","SDIV","AVL","FHLC","CGXU","IBDU","LVHI","IBDS","SCHK","BUFD","DFAX","TLH","JMST","FJP","IBDR","DFIV","DGRW","FIXD","RYLD","QDTE","EVTR","FLCB","BBIN","JMEE","WGMI","SPGP",
          "HYMB","JPIE","IWN",]

EU_ETF = ["BX4.PA","BXX.PA","WPEA.PA","DSD.PA","AUEM.PA","MSE.PA","ESE.PA","SHC.PA","AEEM.PA","MFEC.PA","DFND-EUR.PA","BNKE.PA","LVC.PA","500U.PA","LCWD.PA","PSP5.PA","GRE.PA","ESEH.PA","CL2.PA","ISRA.PA","BNK.PA","ETZ.PA","PASI.PA","SEME.PA","ETZD.PA",
          "CNY.PA","PAASI.PA","PSPH.PA","ESD.PA","ABDJI.PA","ABNSP.PA","AABCH.PA","AATCX.PA","AAHLT.PA","ABSMI.PA","CACC.PA","PUST.PA","AAFIN.PA","AABFB.PA","GEMU.PA","AARTL.PA","OBLI.PA","8G19V.PA","NB22V.PA","NB28V.PA","SPEEU.PA","ERTH.PA","AABEN.PA",
          "ABDJE.PA","ABHSN.PA","AAUTL.PA","AAINS.PA","HSTE.PA","SP5.PA","AASTX.PA","ABNSQ.PA","BRES.PA","PE500.PA","HEMA.PA","AABKX.PA","PAEEM.PA","EEE.PA","AATLC.PA","E40.PA","MUSRI.PA","ESDD.PA","XQ48V.PA","CAC.PA","UST.PA","EEMK.PA","CEC.PA",
          "EGRI.PA","ETHC-EUR.PA","0G28.IL","0Y4H.IL","0MPR.IL","WDTE.L","0MPY.IL","TS3S.L","0A09.L","XT2D.L","0DZF.IL","CNYA.L","SUOE.L","FRXD.L","WHCE.L","XSD2.L","0HOV.IL","ISF.L","JCAU.L","N100.L","0Y8R.IL",
          "DTLA.L","0MRJ.IL","IBTA.L","SUSW.L","SPEH.L","HIGG.L","0XC5.IL","HAGG.L","UC76.L","FBTC.L","AGBP.L","MIDD.L","FUSR.L","0DZB.L","DTLE.L","0MMG.IL","IGLT.L","0Y2B.L","SDIA.L","IEBB.IL","0GBX.IL","JCPN.L",
          "IHYA.L","EWSX.L","0L4R.L","HYLA.L","IDTL.L","EWSP.L","IUIT.L","WNDI.L","0DYZ.IL","0HEP.L","PAWD.L","HCGU.L","0MP3.IL","SNGB.L","0WA3.IL","MTHG.L","FLOA.L","EMAU.L","LQDA.L","HGAS.L","IEAC.L",
          "V3SU.L","AREG.L","0H9U.L","NDIA.L","FWRG.L","IGTM.L","TSLQ.L","GDMS.L","0WA5.IL","UIFS.L","XGSI.L","0KZC.L","BS0A.L","XDNS.L","I50D.L","QQQ5.L"]

CURRENCY_AND_FUTURES = ["ZN=F","ZF=F","ES=F","ZT=F","NQ=F","ZB=F","CL=F","GC=F","ZC=F","RTY=F","NG=F","MGC=F","YM=F","ZS=F","HG=F","SI=F","SB=F","HO=F","RB=F","ZL=F","KE=F","ZM=F","BZ=F","CT=F","KC=F","PL=F","LE=F","SIL=F",
                        "HE=F","CC=F","GF=F","PA=F","EURUSD=X","JPY=X","GBPUSD=X","AUDUSD=X","NZDUSD=X","EURJPY=X","GBPJPY=X","EURGBP=X","EURCAD=X","EURSEK=X","EURCHF=X","EURHUF=X","CNY=X","HKD=X",
                        "SGD=X","INR=X","MXN=X","PHP=X","IDR=X","THB=X","MYR=X","ZAR=X","RUB=X"]

US_TICKERS = [
    #US main tickers
'NVDA','AAPL','MSFT','AMZN','GOOGL','GOOG','META','TSLA','AVGO','BRK-B','ORCL','TCEHY','TCTZF','NFLX','COST','NONOF','LVMHF','LVMUY','JPM-PD','JPM-PC','BML-PG','SAPGF','BML-PH','BML-PL','BAC-PE','IDCBY','BAC-PK','SSNLF','ABBV','IDCBF','HESAY','ASMLF','ASML','GDVTZ',
    'ACGBF','TMUS','BML-PJ','BAC-PB','RHHBF','CSCO','RHHVF','RHHBY','TOYOF','ACGBY','BABAF','AZNCF','BABA','NSRGY','NSRGF','ISRG','CICHY','RYDAF','BACHY','LRLCY','WFC-PY','NVSEF','BACHF','SHEL','CICHF','LRLCF','PCCYF','ADBE','QCOM','HBCYF','HSBC','CMWAY','PLTR','SIEGY',
    'SMAWF','INTU','ANET','CBAUF','IDEXF','SBGSF','SPGI','IDEXY','SBGSY','MBFJF','DTEGF','AMAT','DTEGY','FMXUF','SCHW','CIHKY','MUFG','AMGN','UBER','CMCSA','UNLYF','SHOP','EADSF','EADSY','BHPLF','SNYNF','TTFNF','SNEJF','CHDRY','CILJF','SONY','CHDRF',
    'WFC-PC','ALIZF','ALIZY','HTHIF','ESLOY','RTNTF','HTHIY','MPNGF','Meituan','ESLOF','CIHHF','XIACY','MPNGY','Meituan','PNGAY','XIACF','CIIHF','GILD','PIAIF','VRTX','BYDDF','BYDDY','SBUX','SAFRF','SAFRY','CFRUY','CUAEF','CFRHF','ABBNY','ABLZF','MRVL','KYCCF','RCRUY',
    'UNCFF','UNCRY','CSUAY','SPOT','FRCOF','RCRRF','LRCX','KLAC','SMFNF','SNPMF','SFTBF','FRCOY','AIQUY','AIQUF','SFTBY','SMFG','RTPPF','BUDFF','USB-PH','IBKR','CRWD','DBSDY','RLXXF','EQIX','RELX','INTC','INFY','IVSXF','PYPL','EBBNF','DBSDF','PROSF','PROSY','GS-PA','IBDRY',
    'IBDSF','CDNS','NPPXF','IVSBF','ZFSVF','MS-PA','ZURVY','WELL','BNPQF','BNPQY','SNPS','ATLCY','TOELF','BPAQF','AXAHF','AXAHY','CSLLY','MSTR','CMXHF','ATLKY','MS-PK','TOELY','GS-PD','MS-PI','PBR-A','NTTYY','MS-PF','IITSF','BTAFF','BCDRF','DELL','CTAS','ABNB','MS-PE','AAIGF',
    'LNSTY','LDNXF','ISNPY','RACE','MDLZ','HNHPF','AAGIY','SCCO','DASH','CGXYY','PBCRF','USB-PP','COIN','NTDOF','SBKFF','FTNT','REGN','NTDOY','MURGY','MURGF','PBCRY','ESOCF','WEBNF','ENLAY','NABZY','BKFCF','PSTVY','GLAXF','BCMXY','TEAM','CHGCY','TKOMF','CHGCF','PSBKF','WDAY',
    'ITOCF','RLLCF','MKGAF','NTES','MKKGY','SHECY','BBVXF','ITOCY','STOHF','RBSPF','TKOMY','DGEAF','EQNR','PPWLM','ADSK','BBVA','RYCEF'
]

# Liste de tickers (sans leverage)
TICKERS =['^GSPC','^FCHI','MC.PA', 'OR.PA', 'SU.PA', 'AIR.PA', 'TTE.PA', 'SAN.PA', 'CDI.PA', 'EL.PA', 'SAF.PA', 'AI.PA', 'BNP.PA', 'CS.PA', 'DG.PA', 'DSY.PA', 'SGO.PA', 'BN.PA', 'ACA.PA', 'ENGI.PA', 'KER.PA', 'HO.PA', 'CAP.PA', 'RI.PA', 'LR.PA', 'ORA.PA', 'PUB.PA',
          'GLE.PA', 'ML.PA', 'DIM.PA', 'VIE.PA', 'AM.PA', 'BOL.PA', 'RNO.PA', 'BVI.PA', 'AMUN.PA', 'BIM.PA', 'AC.PA', 'EN.PA', 'ENX.PA', 'ADP.PA', 'URW.PA', 'SW.PA', 'IPN.PA', 'RNL.PA', 'ERF.PA', 'ALO.PA', 'CA.PA', 'FGR.PA', 'LI.PA', 'GET.PA', 'EDEN.PA',
          'RXL.PA', 'IAM.PA', 'CBDG.PA', 'GFC.PA', 'FDJ.PA', 'ODET.PA', 'COTY.PA', 'AKE.PA', 'AYV.PA', 'RF.PA', 'COV.PA', 'GTT.PA', 'SPIE.PA', 'TEP.PA', 'SK.PA', 'TE.PA', 'ELIS.PA', 'SCR.PA', 'VK.PA', 'NEX.PA', 'MF.PA', 
          'MLHK.PA', 'TKO.PA', 'FLY.PA', 'SOP.PA', 'DEC.PA', 'PLX.PA', 'ITP.PA', 'VRLA.PA', 'SOI.PA', 'RCO.PA', 'COVH.PA', 'MMB.PA', 'ATE.PA', 'VU.PA', 'FR.PA', 'IDL.PA', 'BB.PA', 'RUI.PA', 'VIRP.PA', 'TRI.PA', 'BAIN.PA', 'VIV.PA', 'COFA.PA', 'CARM.PA',
          'LOUP.PA', 'NK.PA', 'WLN.PA', 'ALTA.PA', 'UNBL.PA', 'IPS.PA', 'CAF.PA', 'AF.PA', 'PLNW.PA', 'CBE.PA', 'VCT.PA', 'PEUG.PA', 'RBT.PA', 'EXN.PA', 'STF.PA', 'ICAD.PA', 'SESG.PA', 'OPM.PA', 'ERA.PA', 'ARG.PA', 'TFI.PA', 'UBI.PA', 'OVH.PA',
          'MMT.PA', 'ES.PA', 'BLV.PA', 'MAU.PA', 'GDS.PA', 'FII.PA', 'WAVE.PA', 'CRLA.PA', 'NRO.PA', 'LSS.PA', 'MERY.PA', 'ETL.PA', 'ELEC.PA', 'FREY.PA', 'DBG.PA', 'CNDF.PA', 'LTA.PA', 'CDA.PA', 'FNAC.PA', 'MTU.PA', 'VETO.PA', 'TKTT.PA', 'VIL.PA', 'BEN.PA',
          'EC.PA', 'VAC.PA', 'SAVE.PA', 'BASS.PA', 'NXI.PA', 'XFAB.PA', 'SDG.PA', 'THEP.PA', 'CRAV.PA', 'CRSU.PA', 'CRAP.PA', 'CEN.PA', 'SCHP.PA', 'TFF.PA', 'KOF.PA', 'QDT.PA', 'LPE.PA', 'SBT.PA', 'EQS.PA', 'BUR.PA', 'AUB.PA', 'GLO.PA', 'NVDA', 'AAPL', 'MSFT',
          'AMZN', 'GOOGL', 'GOOG', 'META', 'TSLA', 'AVGO', 'BRK-B', 'ORCL', 'TCEHY', 'TCTZF', 'NFLX', 'COST', 'NONOF', 'LVMHF', 'LVMUY', 'JPM-PD', 'JPM-PC', 'BML-PG', 'SAPGF', 'BML-PH', 'BML-PL', 'BAC-PE', 'IDCBY', 'BAC-PK', 'SSNLF', 'ABBV', 'IDCBF', 'HESAY',
          'ASMLF', 'ASML', 'GDVTZ', 'ACGBF', 'TMUS', 'BML-PJ', 'BAC-PB', 'RHHBF', 'CSCO', 'RHHVF', 'RHHBY', 'TOYOF', 'ACGBY', 'BABAF', 'AZNCF', 'BABA', 'NSRGY', 'NSRGF', 'ISRG', 'CICHY', 'RYDAF', 'BACHY', 'LRLCY', 'WFC-PY', 'NVSEF', 'BACHF', 'SHEL', 'CICHF',
          'LRLCF', 'PCCYF', 'ADBE', 'QCOM', 'HBCYF', 'HSBC', 'CMWAY', 'PLTR', 'SIEGY', 'SMAWF', 'INTU', 'ANET', 'CBAUF', 'IDEXF', 'SBGSF', 'SPGI', 'IDEXY', 'SBGSY', 'MBFJF', 'DTEGF', 'AMAT', 'DTEGY', 'FMXUF', 'SCHW', 'CIHKY', 'MUFG', 'AMGN', 'UBER', 'CMCSA',
          'UNLYF', 'SHOP', 'EADSF', 'EADSY', 'BHPLF', 'SNYNF', 'TTFNF', 'SNEJF', 'CHDRY', 'CILJF', 'SONY', 'CHDRF', 'WFC-PC', 'ALIZF', 'ALIZY', 'HTHIF', 'ESLOY', 'RTNTF', 'HTHIY', 'MPNGF', 'ESLOF', 'CIHHF', 'XIACY', 'MPNGY', 'PNGAY', 'XIACF', 'CIIHF', 'GILD',
          'PIAIF', 'VRTX', 'BYDDF', 'BYDDY', 'SBUX', 'SAFRF', 'SAFRY', 'CFRUY', 'CUAEF', 'CFRHF', 'ABBNY', 'ABLZF', 'MRVL', 'KYCCF', 'RCRUY', 'UNCFF', 'UNCRY', 'CSUAY', 'SPOT', 'FRCOF', 'RCRRF', 'LRCX', 'KLAC', 'SMFNF', 'SNPMF', 'SFTBF', 'FRCOY', 'AIQUY',
          'AIQUF', 'SFTBY', 'SMFG', 'RTPPF', 'BUDFF', 'USB-PH', 'IBKR', 'CRWD', 'DBSDY', 'RLXXF', 'EQIX', 'RELX', 'INTC', 'INFY', 'IVSXF', 'PYPL', 'EBBNF', 'DBSDF', 'PROSF', 'PROSY', 'GS-PA', 'IBDRY', 'IBDSF', 'CDNS', 'NPPXF', 'IVSBF', 'ZFSVF', 'MS-PA',
          'ZURVY', 'WELL', 'BNPQF', 'BNPQY', 'SNPS', 'ATLCY', 'TOELF', 'BPAQF', 'AXAHF', 'AXAHY', 'CSLLY', 'MSTR', 'CMXHF', 'ATLKY', 'MS-PK', 'TOELY', 'GS-PD', 'MS-PI', 'PBR-A', 'NTTYY', 'MS-PF', 'IITSF', 'BTAFF', 'BCDRF', 'DELL', 'CTAS', 'ABNB', 'MS-PE',
          'AAIGF', 'LNSTY', 'LDNXF', 'ISNPY', 'RACE', 'MDLZ', 'HNHPF', 'AAGIY', 'SCCO', 'DASH', 'CGXYY', 'PBCRF', 'USB-PP', 'COIN', 'NTDOF', 'SBKFF', 'FTNT', 'REGN', 'NTDOY', 'MURGY', 'MURGF', 'PBCRY', 'ESOCF', 'WEBNF', 'ENLAY', 'NABZY', 'BKFCF', 'PSTVY',
          'GLAXF', 'BCMXY', 'TEAM', 'CHGCY', 'TKOMF', 'CHGCF', 'PSBKF', 'WDAY', 'ITOCF', 'RLLCF', 'MKGAF', 'NTES', 'MKKGY', 'SHECY', 'BBVXF', 'ITOCY', 'STOHF', 'RBSPF', 'TKOMY', 'DGEAF', 'EQNR', 'PPWLM', 'ADSK', 'BBVA', 'RYCEF', 'DBK.DE', 'EI.PA', 'ENEL.MI',
          'FRE.DE', 'IBE.MC', 'INGA.AS', 'ISP.MI', 'EOAN.DE', 'G.MI', 'ALV.DE', 'BBVA.MC', 'BAYN.DE', 'ABI.BR', 'ENI.MI', 'BMW.DE', 'ASML.AS', 'DTE.DE', 'BAS.DE', 'EURUSD=X', 'MT.AS', 'RMS.PA', 'STLAP.PA', 'STMPA.PA', 'V', 'JPM', 'JNJ', 'WMT', 'PG', 'MA',
          'DIS', 'HD', 'VZ', 'UNH', 'KO', 'PFE', 'XOM', 'MRK', 'NKE', 'ABT', 'PEP', 'CRM', 'TMO', 'MDT', 'LLY', 'MS', 'BA', 'STLA', 'NESN.SW', 'A', 'AAL', 'AAP', 'ACN', 'ADI', 'ADM', 'ADP', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM',
          'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMD', 'AME', 'AMP', 'AMT', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'AVB', 'AVY', 'AWK', 'AXP', 'AZO', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR',
          'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CE', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'CPB', 'CPRT', 'CSX', 'CVS',
          'CVX', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DLR', 'DLTR', 'DOV', 'DOW', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'EMN', 'EMR', 'EOG', 'EQR', 'ES', 'ESS', 'ETN', 'ETR', 'EVRG',
          'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FANG', 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FIS', 'FITB', 'FLR', 'FLS', 'FMC', 'FOX', 'FOXA', 'FTV', 'GD', 'GE', 'GIS', 'GL', 'GLW', 'GM', 'GPC', 'GPN', 'GS', 'GWW', 'HAL', 'HAS', 'HBAN', 'HBI', 'HCA', 'HCP',
          'HES', 'HIG', 'HII', 'HLT', 'HOG', 'HOLX', 'HON', 'HP', 'HPE', 'HPQ', 'HRB', 'HRL', 'HSIC', 'HST', 'HSY', 'HUM', 'IBM', 'ICE', 'IDXX', 'IEX', 'IFF', 'ILMN', 'INCY', 'INFO', 'IP', 'IPG', 'IPGP', 'IQV', 'IR', 'IRM', 'IT', 'ITW', 'IVZ', 'J', 'JBHT',
          'JCI', 'JKHY', 'JNPR', 'JWN', 'K', 'KEY', 'KEYS', 'KHC', 'KIM', 'KMB', 'KMI', 'KMX', 'KR', 'KSS', 'L', 'LB', 'LDOS', 'LEG', 'LEN', 'LH', 'LHX', 'LIN', 'LKQ', 'LMT', 'LNC', 'LNT', 'LOW', 'LUV', 'LW', 'LYB', 'M', 'MAA', 'MAC', 'MAR', 'MAS', 'MCD',
          'MCHP', 'MCK', 'MCO', 'MET', 'MGM', 'MHK', 'MKC', 'MKTX', 'MLM', 'MMC', 'MMM', 'MNST', 'MO', 'MOS', 'MPC', 'MSCI', 'MSI', 'MTB', 'MTD', 'MU', 'NAVI', 'NCLH', 'NDAQ', 'NEE', 'NEM', 'NI', 'NOC', 'NOV', 'NRG', 'NSC', 'NTAP', 'NTRS', 'NUE', 'NVR', 'NWL',
          'NWS', 'NWSA', 'O', 'ODFL', 'OKE', 'OMC', 'ORLY', 'OXY', 'PAYC', 'PAYX', 'PCAR', 'PEG', 'PFG', 'PGR', 'PH', 'PHM', 'PLD', 'PM', 'PNC', 'PNR', 'PNW', 'PPG', 'PPL', 'PRGO', 'PRU', 'PSA', 'PSX', 'PVH', 'PWR', 'QRVO', 'RCL', 'REG', 'RF', 'RHI', 'RJF',
          'RL', 'RMD', 'ROK', 'ROL', 'ROP', 'ROST', 'RSG', 'RTX', 'SBAC', 'SEE', 'SHW', 'SJM', 'SLB', 'SLG', 'SNA', 'SO', 'SPG', 'SRE', 'STE', 'STT', 'STX', 'STZ', 'SWK', 'SWKS', 'SYY', 'T', 'TAP', 'TDG', 'TEL', 'TER', 'TFC', 'TFX', 'TGT', 'TJX', 'TPR', 'TRMB', 
          'TROW', 'TRV', 'TSCO', 'TSN', 'TT', 'TTWO', 'TXN', 'TXT', 'TYL', 'UA', 'UAA', 'UAL', 'UDR', 'UHS', 'ULTA', 'UNM', 'UNP', 'UPS', 'URI', 'USB', 'VFC', 'VLO', 'VMC', 'VNO', 'VRSK', 'VRSN', 'VTR', 'WAB', 'WAT', 'WBA', 'WDC', 'WEC', 'WFC', 'WHR', 'WM', 
          'WMB', 'WRB', 'WU', 'WY', 'WYNN', 'XEL', 'XRAY', 'XRX', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION', 'ZTS', 'ADS.DE', 'CON.DE', 'DB1.DE', 'LHA.DE', 'LIN.DE', 'MUV2.DE', 'RWE.DE', 'SAP.DE', 'SIE.DE', 'VOW3.DE', 'ZAL.DE', 'AAL.L', 'ABF.L', 'ADM.L', 'AHT.L', 
          'AV.L', 'BA.L', 'BARC.L', 'BATS.L', 'BP.L', 'BTI', 'CNA.L', 'DGE.L', 'GSK.L', 'HSBA.L', 'IMB.L', 'ITV.L', 'LGEN.L', 'LLOY.L', 'RDSA.VI', 'PUM.DE', 'NOKIA.HE', '2388.HK', '1398.HK', '600519.SS', '601988.SS', '601288.SS', '601318.SS', '002475.SZ', 'BHP.AX', 'CBA.AX', 'TLS.AX', 'WBC.AX', 'CSL.AX', 'NAB.AX', 'ANZ.AX', 'RIO.AX', 'QBE.AX', 'WOW.AX', 'S32.AX', 'FMG.AX', 'MQG.AX', 'TD.TO', 'RY.TO', 'BNS.TO', 'ENB.TO', 'SU.TO', 'CNQ.TO', 'BMO.TO', 'SHOP.TO', 'SLF.TO', 'MFC.TO', 'PPL.TO',
          'TRP.TO', 'TSM', 'SAP', 'VOW.DE', 'ZN=F', 'ZF=F', 'ES=F', 'ZT=F', 'NQ=F', 'ZB=F', 'CL=F', 'GC=F', 'ZC=F', 'RTY=F', 'NG=F', 'MGC=F', 'YM=F', 'ZS=F', 'HG=F', 'SI=F', 'SB=F', 'HO=F', 'RB=F', 'ZL=F', 'KE=F', 'ZM=F', 'BZ=F', 'CT=F', 'KC=F', 'PL=F', 'LE=F', 
          'SIL=F', 'HE=F', 'CC=F', 'GF=F', 'PA=F', 'EURUSD=X', 'JPY=X', 'GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X', 'EURJPY=X', 'GBPJPY=X', 'EURGBP=X', 'EURCAD=X', 'EURSEK=X', 'EURCHF=X', 'EURHUF=X', 'CNY=X', 'HKD=X', 'SGD=X', 'INR=X', 'MXN=X', 'PHP=X', 'IDR=X', 'THB=X',
          'MYR=X', 'ZAR=X', 'RUB=X', 'BX4.PA', 'BXX.PA', 'WPEA.PA', 'DSD.PA', 'AUEM.PA', 'MSE.PA', 'ESE.PA', 'SHC.PA', 'AEEM.PA', 'MFEC.PA', 'DFND-EUR.PA', 'BNKE.PA', 'LVC.PA', '500U.PA', 'LCWD.PA', 'PSP5.PA', 'GRE.PA', 'ESEH.PA', 'CL2.PA', 'ISRA.PA', 'BNK.PA', 
          'ETZ.PA', 'PASI.PA', 'SEME.PA', 'ETZD.PA', 'CNY.PA', 'PAASI.PA', 'PSPH.PA', 'ESD.PA', 'ABDJI.PA', 'ABNSP.PA', 'AABCH.PA', 'AATCX.PA', 'AAHLT.PA', 'ABSMI.PA', 'CACC.PA', 'PUST.PA', 'AAFIN.PA', 'AABFB.PA', 'GEMU.PA', 'AARTL.PA', 'OBLI.PA', '8G19V.PA', 
          'NB22V.PA', 'NB28V.PA', 'SPEEU.PA', 'ERTH.PA', 'AABEN.PA', 'ABDJE.PA', 'ABHSN.PA', 'AAUTL.PA', 'AAINS.PA', 'HSTE.PA', 'SP5.PA', 'AASTX.PA', 'ABNSQ.PA', 'BRES.PA', 'PE500.PA', 'HEMA.PA', 'AABKX.PA', 'PAEEM.PA', 'EEE.PA', 'AATLC.PA', 'E40.PA', 'MUSRI.PA',
          'ESDD.PA', 'XQ48V.PA', 'CAC.PA', 'UST.PA', 'EEMK.PA', 'CEC.PA', 'EGRI.PA', 'ETHC-EUR.PA', '0G28.IL', '0Y4H.IL', '0MPR.IL', 'WDTE.L', '0MPY.IL', 'TS3S.L', '0A09.L', 'XT2D.L', '0DZF.IL', 'CNYA.L', 'SUOE.L', 'FRXD.L', 'WHCE.L', 'XSD2.L', '0HOV.IL', 'ISF.L',
          'JCAU.L', 'N100.L', '0Y8R.IL', 'DTLA.L', '0MRJ.IL', 'IBTA.L', 'SUSW.L', 'SPEH.L', 'HIGG.L', '0XC5.IL', 'HAGG.L', 'UC76.L', 'FBTC.L', 'AGBP.L', 'MIDD.L', 'FUSR.L', '0DZB.L', 'DTLE.L', '0MMG.IL', 'IGLT.L', '0Y2B.L', 'SDIA.L', 'IEBB.IL', '0GBX.IL', 'JCPN.L',
          'IHYA.L', 'EWSX.L', '0L4R.L', 'HYLA.L', 'IDTL.L', 'EWSP.L', 'IUIT.L', 'WNDI.L', '0DYZ.IL', '0HEP.L', 'PAWD.L', 'HCGU.L', '0MP3.IL', 'SNGB.L', '0WA3.IL', 'MTHG.L', 'FLOA.L', 'EMAU.L', 'LQDA.L', 'HGAS.L', 'IEAC.L', 'V3SU.L', 'AREG.L', '0H9U.L',
          'NDIA.L', 'FWRG.L', 'IGTM.L', 'GDMS.L', '0WA5.IL', 'UIFS.L', 'XGSI.L', '0KZC.L', 'BS0A.L', 'XDNS.L', 'I50D.L',  'FXI', 'SPY', 'KWEB', 'IBIT', 'MSTU', 'XLF', 'EEM', 'TLT', 'HYG', 'EWZ',  'IWM', 'SLV', 'LQD', 'MSTZ', 'GDX',
          'SCHD', 'VEA', 'EFA', 'SCHF', 'XBI', 'KRE', 'XLE', 'XLV', 'BITO', 'USHY', 'IEMG', 'SCHX', 'XLI', 'ARKK', 'ASHR', 'IEFA', 'EMXC', 'GLD', 'RWM', 'VOO', 'VWO', 'MCHI', 'IJH', 'ETHA', 'XLP', 'MSTX', 'JAAA', 'SCHG', 'EWJ', 'XLU', 'SMH', 'IAU',
          'SGOL', 'QYLD', 'XLB', 'XRT', 'NVD', 'SGOV', 'RSP', 'KORU', 'BND', 'BIL', 'AGG', 'BKLN', 'SCHH', 'SPLG', 'UNG', 'EMLC', 'INDA', 'FEZ', 'CONY', 'SCHB', 'SPTI', 'VCIT', 'GOVT', 'IGV', 'MSOS', 'GDXJ', 'SBIT', 'IQLT', 'VXX', 'SHV', 'PSQ', 'EMB',
          'VXUS', 'JEPQ', 'IYR', 'VTEB', 'SPIB', 'IEF', 'MSTY', 'JPST', 'FBTC', 'XLK', 'SPDW', 'EWH', 'JEPI', 'ILF', 'PGX', 'CONL', 'IJR', 'IYZ', 'SPHY', 'CGGR', 'GGLL', 'EWY', 'KOLD', 'SCHP', 'VTI', 'CGDV', 'SHY', 'ITB', 'SILJ', 'IVV', 'SPTL', 
          'SOXX', 'SRLN', 'XHB', 'TIP', 'EZA', 'ETHE', 'XLC', 'FBND', 'VGIT', 'USFR', 'VCRB', 'HEFA', 'GBTC', 'ARKG', 'VGSH', 'XLY', 'SJNK', 'ACWI', 'ICLN', 'VTV', 'AMZU', 'MAGS', 'GLDM', 'XOP', 'JCPB', 'VNQ', 'SCHA', 'PFF', 'PAVE', 'PDBC', 'EFV',  
          'BNDX', 'DOG', 'EWT', 'IUSB', 'EZU', 'BITB', 'SPYV', 'JNK', 'BSCT', 'DIA', 'IBB', 'XME', 'RSHO', 'EWW', 'SPSM', 'SCHO', 'EWC', 'VEU', 'CALF', 'SCHM', 'SPYG', 'USO', 'TSLT', 'VGK', 'MUB', 'SPMD', 'COWZ', 'ETH', 'COPX', 'AMLP', 'URA', 
          'KBWB', 'PULS', 'EWA', 'BSV', 'IDEV', 'EWG', 'CLOZ', 'VMBS', 'SPTS', 'IXUS', 'SVIX', 'VIXY', 'DFAC', 'SDVY', 'TFLO', 'SCHZ', 'MLPX', 'SPEM', 'SPSB', 'LABU', 'DYNF', 'IGSB', 'CGUS', 'ETHT', 'BLV', 'BBJP', 'UCO', 'YMAX', 'IAUM', 'VCSH', 'AAAU', 'VGLT',
          'VTWO', 'SPMO', 'TSLR', 'DGRO', 'SPAB', 'KBE', 'AUSF', 'PYLD', 'VCLT', 'NVDU', 'JETS', 'IWD', 'CWEB', 'FNDF', 'SCHR', 'FLOT', 'IGIB', 'BIV', 'JBBB', 'MOAT', 'DFIC', 'MBB', 'URNM', 'TBIL', 'QEFA', 'SCHE', 'PVAL', 'FGD', 'IEUR', 'XLG', 'IVW', 'BUFR',
          'DFSV', 'OUNZ', 'SCHI', 'VFLO',  'DFAI', 'FETH', 'SPLV', 'NVDS', 'GRNY', 'MSFU', 'AAPU', 'IEI', 'EEMA', 'CIBR', 'CGCP', 'ITOT', 'SHYG', 'BSCR', 'IWF', 'RDVY', 'QGRW', 'FPE', 'PXH', 'BAR', 'TFI', 'IWP', 'BINC', 'MTUM', 'UUP', 'ACWX', 'PZA',
          'MINT', 'AAPD', 'YMAG', 'SPLB', 'IWR', 'USMV', 'VUG', 'ACES', 'QUAL', 'IGLB', 'SVXY', 'SCHV', 'BTC', 'IYT', 'AMDY', 'AVGX', 'SPYD', 'MRNY', 'ARKB', 'VIG', 'DFLV', 'TCAF', 'ITA', 'KIE', 'URNJ', 'HYD', 'TSDD', 'SPYI', 'VUSB', 'VTIP', 'FDVV', 'HTRB', 
          'FTSM', 'AMZZ', 'FLTR', 'BIZD', 'PTIR', 'DIVO', 'COWG', 'VSS', 'FBL', 'DFEM', 'EPI', 'IOO', 'AVUV', 'VONG', 'VYM', 'FNDE', 'SPHQ', 'FLRN', 'HYEM', 'MDY', 'SIVR', 'USIG', 'DIVI', 'BUG', 'IWB', 'GSLC', 'UCON', 'CHAU', 'AVDV', 'BOTZ', 'FNDA',
          'IBDQ', 'GUNR', 'PAAA', 'AIQ', 'VWOB', 'SIL', 'DBA', 'SCZ', 'DUHP', 'SVOL', 'AVEM', 'CGMS', 'DFSD', 'HYLB', 'FENY', 'DFCF', 'XSMO', 'AMZY', 'REM', 'SLYV', 'IYW', 'BSCP', 'BSCQ', 'AIYY', 'CGBL', 'DBEF', 'FELC', 'EWU', 'WEAT', 'DFAE', 'USD',
          'CGGO', 'GBIL', 'IUSV', 'EFG', 'BTCZ', 'IBDV', 'SPTM', 'SPYU', 'DPST', 'OIH', 'AMZD', 'DJAN', 'FALN', 'EUFN', 'PFFD', 'METU', 'AIRR', 'ETHD', 'DFAU', 'CETH', 'VFH', 'FBCG', 'IDV', 'BSCS', 'GSY', 'ICSH', 'ESGE', 'JGRO', 'DBC', 'CTA', 'FIAT', 'SDIV',
          'AVL', 'FHLC', 'CGXU', 'IBDU', 'LVHI', 'IBDS', 'SCHK', 'BUFD', 'DFAX', 'TLH', 'JMST', 'FJP', 'IBDR', 'DFIV', 'DGRW', 'FIXD', 'RYLD', 'QDTE', 'EVTR', 'FLCB', 'BBIN', 'JMEE', 'WGMI', 'SPGP', 'HYMB', 'JPIE', 'IWN','RR.L']
#Cleaning des tickers vides :
TICKERS = [ticker.strip() for ticker in TICKERS if ticker and ticker.strip()]


import yfinance as yf
import pandas as pd
import os
import json
import time
import logging
import random
from datetime import datetime, timedelta
from tqdm import tqdm
from typing import Dict, List, Optional, Tuple
import numpy as np
from pathlib import Path
import concurrent.futures

# Configuration centralisée
class Config:
    INTERVALS = ['1d', '1h', '15m', '5m', '30m', '1m']
    DATA_DIR = 'parquets'
    TECH_DATA_DIR = 'parquets_technicals'
    METADATA_FILE = 'tickers_metadata.json'
    DASHBOARD_FILE = 'dashboard.html'
    
    UPDATE_THRESHOLDS = {
        '1m': timedelta(minutes=30),
        '5m': timedelta(hours=1),
        '15m': timedelta(hours=2),
        '30m': timedelta(hours=4),
        '1h': timedelta(hours=8),
        '1d': timedelta(days=2)
    }
    
    BATCH_SIZES = {
        '1m': 10, '5m': 15, '15m': 20, '30m': 25, '1h': 30, '1d': 50
    }
    
    EARNINGS_EXCLUDE_PATTERNS = ['.PA', '.L', '.MC', '.AS', '.DE', '.BR', '.VI', 
                                '.MI', '.HE', '.SW', '.AX', '.TO', '.HK', '.SS', 
                                '.SZ', '=F', '=X', '^', 'ETF']


class ErrorClassifier:
    """Classifier les erreurs pour éviter les blacklistings inutiles"""
    
    PERMANENT_ERROR_PATTERNS = [
        "possibly delisted",
        "symbol may be delisted", 
        "No data found, symbol may be delisted"
    ]
    
    TEMPORAL_ERROR_PATTERNS = [
        "data not available for startTime",
        "The requested range must be within the last"
    ]
    
    @classmethod
    def classify_error(cls, error_message: str) -> str:
        """
        Classifie les erreurs en catégories pour traitement approprié
        Returns: 'permanent', 'temporal', 'temporary', 'network'
        """
        error_lower = error_message.lower()
        
        # Erreurs permanentes - ticker délisté ou inexistant
        if any(pattern in error_lower for pattern in cls.PERMANENT_ERROR_PATTERNS):
            return 'permanent'
        
        # Erreurs temporelles - limitations Yahoo Finance
        if any(pattern in error_lower for pattern in cls.TEMPORAL_ERROR_PATTERNS):
            return 'temporal'
        
        # Erreurs réseau/temporaires
        if any(term in error_lower for term in ['timeout', 'connection', 'network']):
            return 'network'
        
        return 'temporary'

class SmartCache:
    def __init__(self, max_size: int = 1000, ttl_hours: int = 24):
        self.max_size = max_size
        self.ttl = timedelta(hours=ttl_hours)
        self._cache = {}
        self._access_times = {}
        self._hit_count = 0
        self._miss_count = 0
        
    def get(self, key: str, last_modified: float = None) -> Optional[pd.DataFrame]:
        if key not in self._cache:
            self._miss_count += 1
            return None
            
        cached_data, cached_time, cached_mtime = self._cache[key]
        
        if datetime.now() - cached_time > self.ttl:
            del self._cache[key]
            del self._access_times[key]
            self._miss_count += 1
            return None
            
        if last_modified and cached_mtime < last_modified:
            del self._cache[key]
            del self._access_times[key]
            self._miss_count += 1
            return None
            
        self._access_times[key] = datetime.now()
        self._hit_count += 1
        return cached_data.copy()
    
    def put(self, key: str, data: pd.DataFrame, last_modified: float = None):
        while len(self._cache) >= self.max_size:
            oldest_key = min(self._access_times.keys(), key=lambda k: self._access_times[k])
            del self._cache[oldest_key]
            del self._access_times[oldest_key]
            
        self._cache[key] = (data.copy(), datetime.now(), last_modified or time.time())
        self._access_times[key] = datetime.now()
    
    def get_stats(self) -> Dict:
        total = self._hit_count + self._miss_count
        return {
            'hit_rate': (self._hit_count / total * 100) if total > 0 else 0,
            'size': len(self._cache),
            'hits': self._hit_count,
            'misses': self._miss_count
        }

class MetadataManager:
    def __init__(self, error_window_days=30):
        self.data = self._load()
        self.logger = logging.getLogger("MetadataManager")
        self.error_window_days = error_window_days
        
    def _load(self) -> Dict:
        if os.path.exists(Config.METADATA_FILE):
            try:
                with open(Config.METADATA_FILE, 'r') as f:
                    return json.load(f)
            except:
                return {}
        return {}
    
    def save(self):
        with open(Config.METADATA_FILE, 'w') as f:
            json.dump(self.data, f, indent=2)
    
    def get_ticker_info(self, ticker: str) -> Dict:
        return self.data.get(ticker, {
            'last_updates': {},
            'error_timestamps': [],
            'consecutive_errors': 0,
            'last_attempt': None,
            'sector': None,
            'sector_last_updated': None,
            'earnings_last_updated': None,
            'earnings_available': None,
            'blacklisted_until': None
        })
    
    def update_success(self, ticker: str, interval: str):
        if ticker not in self.data:
            self.data[ticker] = self.get_ticker_info(ticker)
        
        self.data[ticker]['last_updates'][interval] = datetime.now().isoformat()
        self.data[ticker]['consecutive_errors'] = 0
        self.data[ticker]['blacklisted_until'] = None
    

    def update_sector(self, ticker: str, sector: str):
        if ticker not in self.data:
            self.data[ticker] = self.get_ticker_info(ticker)
        
        self.data[ticker]['sector'] = sector
        self.data[ticker]['sector_last_updated'] = datetime.now().isoformat()
    
    def update_earnings_attempt(self, ticker: str, available: bool):
        if ticker not in self.data:
            self.data[ticker] = self.get_ticker_info(ticker)
        
        self.data[ticker]['earnings_last_updated'] = datetime.now().isoformat()
        self.data[ticker]['earnings_available'] = available

    def _is_earnings_eligible(self, ticker: str) -> bool:
        if not ticker or len(ticker) > 5:
            return False
        
        for pattern in Config.EARNINGS_EXCLUDE_PATTERNS:
            if pattern in ticker:
                return False
        
        return True

    def is_blacklisted(self, ticker: str) -> bool:
        info = self.get_ticker_info(ticker)
        if not info.get('blacklisted_until'):
            return False
        
        blacklist_end = datetime.fromisoformat(info['blacklisted_until'])
        return datetime.now() < blacklist_end
    
    def get_priority_score(self, ticker: str, interval: str) -> Optional[float]:
        """Calcule un score de priorité intelligent basé sur l'âge, les erreurs et le blacklisting"""
        if self.is_blacklisted(ticker):
            return None
        
        info = self.get_ticker_info(ticker)
        
        # Score de base selon l'âge des données
        last_update_str = info['last_updates'].get(interval)
        if last_update_str:
            last_update = datetime.fromisoformat(last_update_str)
            age_hours = (datetime.now() - last_update).total_seconds() / 3600
        else:
            age_hours = float('inf')  # Priorité maximale pour les données jamais téléchargées
        
        # Pénalités intelligentes
        consecutive_errors = info.get('consecutive_errors', 0)
        error_count = len(info.get('error_timestamps', []))
        
        # Réduction du score selon les erreurs
        error_penalty = consecutive_errors * 10 + error_count * 2
        
        # Bonus de priorité selon l'intervalle
        interval_priority = {
            '1m': 100, '5m': 80, '15m': 60, '30m': 40, '1h': 20, '1d': 10
        }
        interval_bonus = interval_priority.get(interval, 0)
        
        # Score final
        final_score = age_hours + interval_bonus - error_penalty
        
        return max(final_score, 0)  # Pas de score négatif
    
    def needs_update(self, ticker: str, interval: str) -> bool:
        if self.is_blacklisted(ticker):
            return False
            
        info = self.get_ticker_info(ticker)
        last_update_str = info['last_updates'].get(interval)
        
        if not last_update_str:
            return True
            
        last_update = datetime.fromisoformat(last_update_str)
        threshold = Config.UPDATE_THRESHOLDS.get(interval, timedelta(days=1))
        
        return (datetime.now() - last_update) > threshold
    
    def should_update_earnings(self, ticker: str) -> bool:
        if not self._is_earnings_eligible(ticker):
            return False
            
        info = self.get_ticker_info(ticker)
        
        if not info.get('earnings_last_updated'):
            return True
        
        last_attempt = datetime.fromisoformat(info['earnings_last_updated'])
        return (datetime.now() - last_attempt).days >= 90
    
    def should_update_sector(self, ticker: str) -> bool:
        info = self.get_ticker_info(ticker)
        
        if not info.get('sector') or info['sector'] == 'Unknown':
            return True
        
        if not info.get('sector_last_updated'):
            return True
        
        last_update = datetime.fromisoformat(info['sector_last_updated'])
        return (datetime.now() - last_update).days >= 90
       
    def get_known_sectors(self) -> Dict[str, str]:
        sectors = {}
        for ticker, info in self.data.items():
            if info.get('sector') and info['sector'] != 'Unknown':
                sectors[ticker] = info['sector']
        return sectors
    
        
    def update_failure(self, ticker: str, error_message: str = ""):
        """Mise à jour intelligente avec classification d'erreurs améliorée"""
        now = datetime.now()
        if ticker not in self.data:
            self.data[ticker] = self.get_ticker_info(ticker)
        
        info = self.data[ticker]
        error_type = ErrorClassifier.classify_error(error_message)
        
        if error_type == 'permanent':
            # Blacklist permanent pour tickers délistés
            blacklist_until = now + timedelta(days=365)
            info['blacklisted_until'] = blacklist_until.isoformat()
            info['delisted'] = True
            info['error_classification'] = 'permanent'
            self.logger.warning(f"Permanently blacklisted {ticker} (delisted)")
            return
        
        elif error_type == 'temporal':
            # Limitation Yahoo Finance - blacklist court avec marquage spécial
            blacklist_until = now + timedelta(hours=48)  # Augmenté à 48h
            info['blacklisted_until'] = blacklist_until.isoformat()
            info['error_classification'] = 'temporal'
            info['temporal_limitation_detected'] = now.isoformat()
            self.logger.info(f"Temporal blacklist for {ticker} (48h - Yahoo limitation)")
            return
        
        # Gestion standard pour erreurs temporaires
        info['error_timestamps'].append(now.isoformat())
        info['consecutive_errors'] += 1
        info['last_attempt'] = now.isoformat()
        info['error_classification'] = error_type
        
        # Nettoyage erreurs anciennes
        cutoff = now - timedelta(days=self.error_window_days)
        info['error_timestamps'] = [
            ts for ts in info['error_timestamps'] 
            if datetime.fromisoformat(ts) >= cutoff
        ]
        
        # Blacklisting progressif amélioré
        error_count = len(info['error_timestamps'])
        consecutive = info['consecutive_errors']
        
        if consecutive >= 3:
            # Formule progressive plus intelligente
            base_hours = min(10 * (consecutive - 2), 168)  # Max 1 semaine
            error_factor = min(error_count / 10, 2)
            total_hours = int(base_hours * (1 + error_factor))
            
            blacklist_until = now + timedelta(hours=total_hours)
            info['blacklisted_until'] = blacklist_until.isoformat()
            
            self.logger.warning(
                f"Progressive blacklist for {ticker}: {total_hours}h "
                f"(consecutive: {consecutive}, recent: {error_count}, type: {error_type})"
            )
    
    def _mark_permanently_delisted(self, ticker: str):
        """Marque un ticker comme définitivement délisté"""
        if ticker not in self.data:
            self.data[ticker] = self.get_ticker_info(ticker)
        
        # Blacklist permanent (10 ans)
        blacklist_until = datetime.now() + timedelta(days=3650)
        self.data[ticker]['blacklisted_until'] = blacklist_until.isoformat()
        self.data[ticker]['delisted'] = True
        self.logger.info(f"Marked {ticker} as permanently delisted")
    
    def _mark_temporal_limitation(self, ticker: str, error_message: str):
        """Gère les limitations temporelles Yahoo Finance"""
        if ticker not in self.data:
            self.data[ticker] = self.get_ticker_info(ticker)
        
        info = self.data[ticker]
        
        # Extraire l'intervalle de l'erreur pour blacklist sélectif
        if "1h data not available" in error_message:
            if 'temporal_limitations' not in info:
                info['temporal_limitations'] = {}
            info['temporal_limitations']['1h'] = datetime.now().isoformat()
            
        self.logger.debug(f"Temporal limitation for {ticker}: {error_message}")

class FileManager:
    def __init__(self, cache: SmartCache):
        self.cache = cache
        self.logger = logging.getLogger("FileManager")
    
    def load_data(self, ticker: str, interval: str) -> Optional[pd.DataFrame]:
        file_path = os.path.join(Config.DATA_DIR, interval, f"{ticker}.parquet")
        
        if not os.path.exists(file_path):
            return None
        
        try:
            last_modified = os.path.getmtime(file_path)
            cached_df = self.cache.get(file_path, last_modified)
            if cached_df is not None:
                return cached_df
            
            df = pd.read_parquet(file_path)
            self.cache.put(file_path, df, last_modified)
            return df
            
        except Exception as e:
            self.logger.warning(f"Error loading {file_path}: {str(e)}")
            return None
    
    def save_data(self, ticker: str, interval: str, new_data: pd.DataFrame, 
                is_technical: bool = False) -> bool:
        if new_data.empty:
            return False
        
        # Validation des colonnes critiques pour les fichiers enrichis
        if not is_technical:
            expected_metadata_cols = ['sector', 'next_earnings_date_to_date', 'last_earnings_date_to_date']
            missing_metadata = [col for col in expected_metadata_cols if col not in new_data.columns]
            if missing_metadata:
                self.logger.debug(f"Metadata columns missing for {ticker}: {missing_metadata}")
        
        # Continuer avec la logique de sauvegarde existante
        base_dir = Config.TECH_DATA_DIR if is_technical else Config.DATA_DIR
        folder_path = os.path.join(base_dir, interval)
        os.makedirs(folder_path, exist_ok=True)
        file_path = os.path.join(folder_path, f"{ticker}.parquet")
        
        try:
            new_data = self._normalize_data(new_data)
            
            if not is_technical:
                existing_data = self.load_data(ticker, interval)
                if existing_data is not None:
                    # === FUSION INTELLIGENTE ===
                    final_data = self._smart_merge(existing_data, new_data)
                    new_rows = len(final_data) - len(existing_data)
                    
                    if new_rows > 0:
                        self.logger.info(f"Added {new_rows} new rows for {ticker} ({interval})")
                    else:
                        self.logger.debug(f"No new data for {ticker} ({interval})")
                else:
                    final_data = new_data
                    self.logger.info(f"Created new file for {ticker} ({interval}): {len(final_data)} rows")
            else:
                final_data = new_data
            
            final_data.to_parquet(file_path, compression='snappy')
            
            # Mettre à jour cache
            last_modified = os.path.getmtime(file_path)
            self.cache.put(file_path, final_data, last_modified)
            
            return True
            
        except Exception as e:
            self.logger.error(f"Error saving {ticker} ({interval}): {str(e)}")
            return False

    def _smart_merge(self, existing_data: pd.DataFrame, new_data: pd.DataFrame) -> pd.DataFrame:
        """Fusion intelligente évitant les doublons"""
        try:
            # Identifier les nouvelles données réelles
            last_existing_date = existing_data.index.max()
            truly_new_data = new_data[new_data.index > last_existing_date]
            
            if truly_new_data.empty:
                return existing_data  # Pas de nouvelles données
            
            # Concaténer et nettoyer
            combined_data = pd.concat([existing_data, truly_new_data])
            combined_data = combined_data[~combined_data.index.duplicated(keep='last')]
            
            return combined_data.sort_index()
            
        except Exception as e:
            self.logger.warning(f"Smart merge failed, using standard merge: {str(e)}")
            # Fallback vers fusion standard
            combined_data = pd.concat([existing_data, new_data])
            combined_data = combined_data[~combined_data.index.duplicated(keep='last')]
            return combined_data.sort_index()
    
    def _normalize_data(self, df: pd.DataFrame) -> pd.DataFrame:
        # Suppression des colonnes parasites - garder uniquement les colonnes valides
        allowed_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 
                          'sector', 'next_earnings_date', 'last_earnings_date', 
                          'days_to_earnings', 'days_from_earnings',
                          'RSI14', 'SMA20', 'SMA50', 'SMA200', 'EMA12', 'EMA26',
                          'Bollinger_Upper', 'Bollinger_Lower', 'Volatility_20', 'Momentum_10']
        
        existing_allowed = [col for col in allowed_columns if col in df.columns]
        df = df[existing_allowed]
        
        df.index = pd.to_datetime(df.index)
        df = df[~df.index.duplicated(keep='last')]  # Supprime les doublons AVANT fusion
        if df.index.tzinfo is not None:
            df.index = df.index.tz_convert('UTC').tz_localize(None)
        return df.sort_index()
    
    def is_fresh(self, ticker: str, interval: str) -> bool:
        file_path = os.path.join(Config.DATA_DIR, interval, f"{ticker}.parquet")
        
        if not os.path.exists(file_path):
            return False
        
        try:
            file_mtime = datetime.fromtimestamp(os.path.getmtime(file_path))
            threshold = Config.UPDATE_THRESHOLDS.get(interval, timedelta(days=1))
            return (datetime.now() - file_mtime) <= threshold
        except:
            return False

class TechnicalIndicatorCalculator:
    def __init__(self):
        self.logger = logging.getLogger("TechnicalCalculator")

    def calculate_all_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calcul optimisé combinant les meilleures méthodes des deux versions"""
        if df.empty or len(df) < 20:
            return df
        
        result_df = df.copy()
        data_length = len(result_df)
        
        try:
            # === NETTOYAGE INITIAL - Garder seulement les colonnes autorisées ===
            allowed_columns = [
                # OHLCV
                'Open', 'High', 'Low', 'Close', 'Volume',
                # Métadonnées
                'sector', 'next_earnings_date_to_date', 'last_earnings_date_to_date', 
                'days_to_next_earnings', 'days_from_last_earnings',
                # Indicateurs techniques (seront ajoutés)
                'RSI14', 'Stochastic_K14', 'Stochastic_D14', 'MRC_Upper', 'MRC_Lower',
                'SMA10', 'SMA20', 'SMA50', 'SMA100', 'SMA200',
                'EMA10', 'EMA20', 'EMA50',
                'Volatility20', 'Volatility50', 'Volatility100', 'Volatility360',
                'Momentum20', 'Momentum50', 'Momentum100', 'Momentum252', 'Momentum360',
                'log_return', 'mu', 'theta', 'sigma_v', 'kappa'
            ]
            
            # Garder seulement les colonnes existantes autorisées
            existing_allowed = [col for col in allowed_columns if col in result_df.columns]
            result_df = result_df[existing_allowed]
            
            close = result_df['Close']
            high = result_df['High'] 
            low = result_df['Low']
            
            # === RSI14 (version optimisée old) ===
            if data_length >= 14:
                def compute_rsi_optimized(series, period=14):
                    series = series.dropna()
                    delta = series.diff()
                    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
                    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
                    rs = gain / loss
                    rsi = 100 - (100 / (1 + rs))
                    return rsi.reindex(series.index)
                
                result_df['RSI14'] = compute_rsi_optimized(close)
            
            # === STOCHASTIQUES (version optimisée old) ===
            if data_length >= 14:
                def compute_stochastic_optimized(high, low, close, k_period=14, d_period=3):
                    high, low, close = high.dropna(), low.dropna(), close.dropna()
                    lowest_low = low.rolling(window=k_period).min()
                    highest_high = high.rolling(window=k_period).max()
                    k = 100 * (close - lowest_low) / (highest_high - lowest_low)
                    d = k.rolling(window=d_period).mean()
                    return k.reindex(close.index), d.reindex(close.index)
                
                k, d = compute_stochastic_optimized(high, low, close)
                result_df['Stochastic_K14'] = k
                if data_length >= 17:
                    result_df['Stochastic_D14'] = d
            
            # === MEAN REVERSION CHANNEL (version v9 - déjà optimisée) ===
            if data_length >= 20:
                rolling_mean = close.dropna().rolling(window=20).mean().reindex(result_df.index)
                rolling_std = close.dropna().rolling(window=20).std().reindex(result_df.index)
                result_df['MRC_Upper'] = rolling_mean + 2 * rolling_std
                result_df['MRC_Lower'] = rolling_mean - 2 * rolling_std
            
            # === SMA (version optimisée old) ===
            for period in [10, 20, 50, 100, 200]:
                if data_length >= period:
                    result_df[f'SMA{period}'] = close.dropna().rolling(window=period).mean().reindex(result_df.index)
            
            # === EMA (version optimisée old) ===
            def compute_ema_optimized(series, span):
                series = series.dropna()
                ema = series.ewm(span=span, adjust=False).mean()
                return ema.reindex(series.index)
            
            for span in [10, 20, 50]:
                if data_length >= span:
                    result_df[f'EMA{span}'] = compute_ema_optimized(close, span)
            
            # === VOLATILITÉ (version optimisée old) ===
            def compute_volatility_optimized(series, window):
                series = series.dropna()
                return series.pct_change().rolling(window=window).std() * np.sqrt(window)
            
            for window in [20, 50, 100, 360]:
                if data_length >= window:
                    result_df[f'Volatility{window}'] = compute_volatility_optimized(close, window)
            
            # === MOMENTUM (version optimisée old avec ffill) ===
            def compute_momentum_optimized(df, periods):
                close_filled = df['Close'].ffill()  # Remplir les NaN en avant
                for period in periods:
                    if len(df) >= period:
                        df[f'Momentum{period}'] = close_filled.pct_change(periods=period) * 100
            
            compute_momentum_optimized(result_df, [20, 50, 100, 252, 360])
            
            # === PARAMÈTRES MONTE CARLO (version v9) ===
            if data_length >= 20:
                result_df['log_return'] = np.log(close / close.shift(1))
                result_df['mu'] = result_df['log_return'].mean()
            
            # === PARAMÈTRES HESTON (version optimisée old) ===
            if data_length >= 360:
                def compute_heston_params_optimized(df):
                    try:
                        volatility_100 = df['Volatility100'].dropna()
                        volatility_360 = df['Volatility360'].dropna()
                        if len(volatility_100) < 10 or len(volatility_360) < 10:
                            return np.nan, np.nan, np.nan
                        ema_volatility_100 = volatility_100.ewm(span=100, adjust=False).mean()
                        ema_volatility_360 = volatility_360.ewm(span=360, adjust=False).mean()
                        theta = np.mean([ema_volatility_100.iloc[-1], ema_volatility_360.iloc[-1]])
                        sigma_v = np.std(volatility_360)
                        kappa = min(max(1 / np.mean(volatility_360.pct_change().dropna()), 0.1), 10)
                        return theta, sigma_v, kappa
                    except Exception as e:
                        self.logger.debug(f"Error computing Heston params: {e}")
                        return np.nan, np.nan, np.nan
                
                theta, sigma_v, kappa = compute_heston_params_optimized(result_df)
                result_df['theta'] = theta
                result_df['sigma_v'] = sigma_v
                result_df['kappa'] = kappa
            
            # === NETTOYAGE FINAL - Supprimer toutes les colonnes non autorisées ===
            final_allowed_columns = [col for col in allowed_columns if col in result_df.columns]
            result_df = result_df[final_allowed_columns]
            
            self.logger.debug(f"Technical indicators calculated: {len(final_allowed_columns)} total columns")
            return result_df
            
        except Exception as e:
            self.logger.warning(f"Error calculating technical indicators: {str(e)}")
            return df

    def needs_calculation(self, ohlcv_file: str, tech_file: str) -> bool:
        """Détermine si le calcul des indicateurs techniques est nécessaire."""
        if not os.path.exists(tech_file):
            return True
        
        try:
            ohlcv_mtime = os.path.getmtime(ohlcv_file)
            tech_mtime = os.path.getmtime(tech_file)
            return ohlcv_mtime > tech_mtime
        except:
            return True
    

class MetadataEnricher:
    def __init__(self, metadata_manager: MetadataManager):
        self.metadata = metadata_manager
        self.logger = logging.getLogger("MetadataEnricher")
        self.earnings_cache = EarningsCalendarCache()
        self.sector_api_calls = 0
        self.earnings_api_calls = 0
    
    def should_update_sector_enricher(self, ticker: str) -> bool:
        """Version enrichie de la vérification de secteur avec logique avancée"""
        # Vérifier d'abord via le gestionnaire de métadonnées
        if not self.metadata.should_update_sector(ticker):
            return False
        
        # Logique additionnelle pour l'enrichissement
        if any(pattern in ticker for pattern in ['^', '=F', '=X', '.PA', '.L']):
            return False
        
        # Limiter les appels API (max 20 par session)
        if self.sector_api_calls >= 20:
            return False
        
        return len(ticker) <= 5

    def enrich_dataframe_advanced(self, df: pd.DataFrame, ticker: str) -> pd.DataFrame:
        """Enrichissement intelligent avec optimisation des appels API"""
        if df.empty:
            return df
        
        result_df = df.copy()
        
        # === INITIALISATION COLONNES MÉTADONNÉES ===
        result_df['sector'] = 'Unknown'
        result_df['next_earnings_date_to_date'] = pd.NaT
        result_df['last_earnings_date_to_date'] = pd.NaT
        result_df['days_to_next_earnings'] = pd.NA
        result_df['days_from_last_earnings'] = pd.NA
        
        # === PROPAGATION SECTEUR INTELLIGENT ===
        cached_sector = self._get_cached_sector(ticker)
        if cached_sector:
            result_df['sector'] = cached_sector
            self.logger.debug(f"Propagated cached sector for {ticker}: {cached_sector}")
        elif self.should_update_sector_enricher(ticker):
            new_sector = self._fetch_sector_with_retry(ticker)
            if new_sector and new_sector != 'Unknown':
                result_df['sector'] = new_sector
                self.metadata.update_sector(ticker, new_sector)
                self.sector_api_calls += 1
            else:
                self.metadata.update_sector(ticker, 'Unknown')
        
        # === EARNINGS INTELLIGENT ===
        if self._is_earnings_eligible_advanced(ticker) and self.metadata.should_update_earnings(ticker):
            try:
                result_df = self._enrich_with_dynamic_earnings(result_df, ticker)
                self.earnings_api_calls += 1
            except Exception as e:
                self.logger.debug(f"Could not retrieve earnings for {ticker}: {str(e)}")
                self.metadata.update_earnings_attempt(ticker, False)
        
        return result_df

    def _should_fetch_sector(self, ticker: str) -> bool:
        """Détermine intelligemment si on doit fetcher le secteur"""
        # Éviter ETFs, indices, futures
        if any(pattern in ticker for pattern in ['^', '=F', '=X', '.L']):
            return False
        
        # Limiter les appels API (max 20 par session)
        if self.sector_api_calls >= 20:
            return False
        
        return len(ticker) <= 5

    def _fetch_sector_with_retry(self, ticker: str, max_retries: int = 2) -> str:
        """Fetch secteur avec retry et gestion d'erreurs"""
        for attempt in range(max_retries):
            try:
                time.sleep(0.5 * (attempt + 1))  # Backoff progressif
                ticker_obj = yf.Ticker(ticker)
                info = ticker_obj.info
                sector = info.get('sector', 'Unknown')
                return sector if sector else 'Unknown'
            except Exception as e:
                if attempt == max_retries - 1:
                    self.logger.debug(f"Failed to fetch sector for {ticker} after {max_retries} attempts")
        
        return 'Unknown'

    def _is_earnings_eligible_advanced(self, ticker: str) -> bool:
        """Détermine l'éligibilité earnings avec logique avancée"""
        if not ticker or len(ticker) > 5:
            return False
        
        # Exclusions intelligentes
        exclude_patterns = Config.EARNINGS_EXCLUDE_PATTERNS + ['ETF', 'FUND', 'REIT']
        if any(pattern in ticker.upper() for pattern in exclude_patterns):
            return False
        
        # Limiter appels API earnings (plus coûteux)
        if self.earnings_api_calls >= 10:
            return False
        
        return True

    def _enrich_with_dynamic_earnings(self, df: pd.DataFrame, ticker: str) -> pd.DataFrame:
        """Enrichissement earnings avec cache intelligent"""
        earnings_calendar = self.earnings_cache.get_earnings_calendar(ticker)
        
        if earnings_calendar is None:
            # Pas en cache, fetch depuis API
            earnings_calendar = self._fetch_full_earnings_calendar(ticker)
            
            if earnings_calendar:
                self.earnings_cache.cache_earnings_calendar(ticker, earnings_calendar)
                self.metadata.update_earnings_attempt(ticker, True)
            else:
                self.metadata.update_earnings_attempt(ticker, False)
                return df
        
        if earnings_calendar:
            df = self._calculate_dynamic_earnings_distances(df, earnings_calendar)
        
        return df

    def _fetch_full_earnings_calendar(self, ticker: str) -> List:
        """Récupère le calendrier complet des earnings"""
        try:
            ticker_obj = yf.Ticker(ticker)
            earnings_dates = ticker_obj.earnings_dates
            
            if earnings_dates is not None and not earnings_dates.empty:
                dates_list = earnings_dates.index.tolist()
                normalized_dates = []
                for date in dates_list:
                    if hasattr(date, 'tz_localize'):
                        date = date.tz_convert('UTC').tz_localize(None)
                    normalized_dates.append(date)
                
                return sorted(normalized_dates)
            
            return []
            
        except Exception as e:
            self.logger.debug(f"Error fetching earnings for {ticker}: {str(e)}")
            return []

    def _calculate_dynamic_earnings_distances(self, df: pd.DataFrame, earnings_calendar: List) -> pd.DataFrame:
        """Calcul dynamique des distances aux earnings"""
        if not earnings_calendar:
            return df
        
        earnings_dates = pd.Series(pd.to_datetime(earnings_calendar)).sort_values()
        
        for date in df.index:
            try:
                # Prochains earnings
                future_earnings = earnings_dates[earnings_dates > date]
                if len(future_earnings) > 0:
                    next_earnings = future_earnings.iloc[0]
                    days_to_next = (next_earnings - date).days
                    
                    if days_to_next <= 120:  # Contexte réaliste
                        df.loc[date, 'next_earnings_date_to_date'] = next_earnings
                        df.loc[date, 'days_to_next_earnings'] = days_to_next
                
                # Derniers earnings
                past_earnings = earnings_dates[earnings_dates <= date]
                if len(past_earnings) > 0:
                    last_earnings = past_earnings.iloc[-1]
                    days_from_last = (date - last_earnings).days
                    
                    if days_from_last <= 120:
                        df.loc[date, 'last_earnings_date_to_date'] = last_earnings
                        df.loc[date, 'days_from_last_earnings'] = days_from_last
                        
            except Exception:
                continue
        
        return df
    
    def propagate_sectors_to_all_intervals(self):
        known_sectors = self.metadata.get_known_sectors()
        
        if not known_sectors:
            self.logger.info("No sectors to propagate")
            return
        
        propagated_count = 0
        
        for ticker, sector in known_sectors.items():
            for interval in Config.INTERVALS:
                file_path = os.path.join(Config.DATA_DIR, interval, f"{ticker}.parquet")
                
                if os.path.exists(file_path):
                    try:
                        df = pd.read_parquet(file_path)
                        
                        if 'sector' not in df.columns or (df['sector'] == 'Unknown').all():
                            df['sector'] = sector
                            df.to_parquet(file_path, compression='snappy')
                            propagated_count += 1
                    
                    except Exception as e:
                        self.logger.warning(f"Error propagating sector for {ticker} ({interval}): {str(e)}")
        
        if propagated_count > 0:
            self.logger.info(f"Propagated sectors to {propagated_count} files")
    
    def update_metadata_from_api(self, tickers: List[str]):
        # Filtrage avec logique avancée pour les secteurs
        tickers_for_sector = [t for t in tickers if self.metadata.should_update_sector(t) 
                            and self._should_fetch_sector(t)]
        
        # Filtrage avec logique avancée pour les earnings
        tickers_for_earnings = [t for t in tickers if self.metadata.should_update_earnings(t) 
                            and self._is_earnings_eligible_advanced(t)]
        
        if tickers_for_sector:
            self._batch_update_sectors(tickers_for_sector)
        
        if tickers_for_earnings:
            self._update_earnings_sequential(tickers_for_earnings)
    
    def _get_cached_sector(self, ticker: str) -> Optional[str]:
        info = self.metadata.get_ticker_info(ticker)
        sector = info.get('sector')
        
        if sector and sector != 'Unknown':
            return sector
        
        return None
    
    def _batch_update_sectors(self, tickers: List[str]):
        self.logger.info(f"Updating sectors for {len(tickers)} tickers")
        
        for ticker in tickers:
            try:
                time.sleep(0.5)
                
                ticker_obj = yf.Ticker(ticker)
                info = ticker_obj.info
                sector = info.get('sector', 'Unknown')
                
                if sector and sector != 'Unknown':
                    self.metadata.update_sector(ticker, sector)
                else:
                    self.metadata.update_sector(ticker, 'Unknown')
            
            except Exception as e:
                self.logger.warning(f"Error updating sector for {ticker}: {str(e)}")
                self.metadata.update_sector(ticker, 'Unknown')
    
    def _update_earnings_sequential(self, tickers: List[str]):
        self.logger.info(f"Updating earnings for {len(tickers)} eligible tickers")
        
        for ticker in tickers:
            try:
                time.sleep(1.0)
                
                earnings_dates = self._fetch_earnings_calendar(ticker)
                has_earnings = len(earnings_dates) > 0
                
                self.metadata.update_earnings_attempt(ticker, has_earnings)
                
                if has_earnings:
                    self.earnings_cache[ticker] = earnings_dates
            
            except Exception as e:
                self.logger.warning(f"Error updating earnings for {ticker}: {str(e)}")
                self.metadata.update_earnings_attempt(ticker, False)
    
    def _fetch_earnings_calendar(self, ticker: str) -> List[datetime]:
        try:
            ticker_obj = yf.Ticker(ticker)
            earnings_dates = ticker_obj.earnings_dates
            
            if earnings_dates is not None and not earnings_dates.empty:
                dates_list = earnings_dates.index.tolist()
                normalized_dates = []
                for date in dates_list:
                    if hasattr(date, 'tz_localize'):
                        date = date.tz_convert('UTC').tz_localize(None)
                    normalized_dates.append(date)
                
                return sorted(normalized_dates)
            
            return []
            
        except Exception:
            return []


class EarningsCalendarCache:
    """Cache global pour les calendriers d'earnings des tickers."""
    
    def __init__(self, cache_file='earnings_calendar_cache.json'):
        self.cache_file = cache_file
        self.cache = self._load_cache()
        self.logger = logging.getLogger("EarningsCache")
        
    def _load_cache(self):
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r') as f:
                    data = json.load(f)
                    # Reconvertir les dates
                    for ticker, info in data.items():
                        if 'earnings_dates' in info:
                            info['earnings_dates'] = [
                                pd.to_datetime(d) for d in info['earnings_dates']
                            ]
                    return data
            except:
                return {}
        return {}
    
    def get_earnings_calendar(self, ticker: str) -> Optional[List]:
        if ticker not in self.cache:
            return None
            
        cached_info = self.cache[ticker]
        # Vérifier TTL (1 semaine)
        if 'cached_at' in cached_info:
            cached_date = datetime.fromisoformat(cached_info['cached_at'])
            if (datetime.now() - cached_date).days > 7:
                return None
                
        return cached_info.get('earnings_dates', [])
    
    def cache_earnings_calendar(self, ticker: str, earnings_dates: List):
        # CORRECTION: Conversion des Timestamps en strings avant sérialisation
        serializable_dates = []
        for date in earnings_dates:
            if hasattr(date, 'isoformat'):
                serializable_dates.append(date.isoformat())
            else:
                serializable_dates.append(str(date))
        
        self.cache[ticker] = {
            'earnings_dates': serializable_dates,
            'cached_at': datetime.now().isoformat()
        }
        
        try:
            with open(self.cache_file, 'w') as f:
                json.dump(self.cache, f, indent=2)
        except Exception as e:
            self.logger.warning(f"Failed to save earnings cache: {str(e)}")


class PeriodOptimizer:
    """Optimise les périodes de téléchargement selon les limitations Yahoo Finance"""
    
    # Limitations connues de Yahoo Finance
    YAHOO_LIMITS = {
        '1m': timedelta(days=7),     # Max 7 jours
        '5m': timedelta(days=60),    # Max 60 jours
        '15m': timedelta(days=60),   # Max 60 jours
        '30m': timedelta(days=60),   # Max 60 jours
        '1h': timedelta(days=730),   # Max 730 jours
        '1d': timedelta(days=365*20) # Max ~20 ans
    }
    
    @classmethod
    def get_safe_period(cls, interval: str, ticker: str = None) -> str:
        """Retourne une période sûre selon l'intervalle et les limitations Yahoo"""
        
        # Périodes optimisées basées sur les limites réelles
        safe_periods = {
            '1m': '5d',      # Réduit de 7d pour marge sécurité
            '5m': '30d',     # Réduit de 60d pour stabilité
            '15m': '30d',    # Plus conservateur
            '30m': '30d',    # Plus conservateur
            '1h': '365d',    # Bien en dessous de 730d
            '1d': '5y'       # Équilibre historique/performance
        }
        
        return safe_periods.get(interval, '1y')
    

    
class DashboardGenerator:
    def __init__(self, metadata_manager: MetadataManager):
        self.metadata = metadata_manager
        self.logger = logging.getLogger("DashboardGenerator")
    
    def generate_dashboard(self, stats: Dict) -> str:
        freshness_data = self._calculate_freshness_metrics()
        problem_tickers = self._get_problem_tickers()
        
        html_content = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Download Dashboard</title>
    <meta http-equiv="refresh" content="300">
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; background: #f9f9f9; }}
        .container {{ max-width: 1000px; margin: 0 auto; }}
        .metrics {{ display: flex; gap: 20px; margin-bottom: 20px; }}
        .metric {{ background: white; padding: 15px; border-radius: 5px; flex: 1; text-align: center; }}
        .metric-value {{ font-size: 24px; font-weight: bold; color: #333; }}
        .metric-label {{ color: #666; margin-top: 5px; }}
        .section {{ background: white; margin-bottom: 20px; padding: 20px; border-radius: 5px; }}
        .section h2 {{ margin: 0 0 15px 0; color: #333; border-bottom: 2px solid #ddd; padding-bottom: 5px; }}
        .status-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(150px, 1fr)); gap: 10px; }}
        .status-item {{ padding: 10px; background: #f5f5f5; border-radius: 3px; }}
        .problems {{ max-height: 200px; overflow-y: auto; }}
        .problem-item {{ padding: 8px; border-bottom: 1px solid #eee; }}
        .timestamp {{ text-align: right; color: #999; font-size: 12px; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Download Dashboard</h1>
        <p class="timestamp">Last update: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        
        <div class="metrics">
            <div class="metric">
                <div class="metric-value">{stats.get('downloads_successful', 0)}</div>
                <div class="metric-label">Successful Downloads</div>
            </div>
            <div class="metric">
                <div class="metric-value">{stats.get('api_calls_saved', 0)}</div>
                <div class="metric-label">API Calls Saved</div>
            </div>
            <div class="metric">
                <div class="metric-value">{stats.get('cache_hit_rate', 0):.1f}%</div>
                <div class="metric-label">Cache Hit Rate</div>
            </div>
            <div class="metric">
                <div class="metric-value">{len(problem_tickers)}</div>
                <div class="metric-label">Problem Tickers</div>
            </div>
        </div>
        
        <div class="section">
            <h2>Status by Interval</h2>
            <div class="status-grid">
                {self._generate_interval_status(freshness_data)}
            </div>
        </div>
        
        {self._generate_problems_section(problem_tickers)}
    </div>
</body>
</html>"""
        
        with open(Config.DASHBOARD_FILE, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        self.logger.info(f"Dashboard generated: {Config.DASHBOARD_FILE}")
        return Config.DASHBOARD_FILE
    
    def _calculate_freshness_metrics(self) -> Dict:
        freshness = {interval: {'fresh': 0, 'stale': 0, 'never': 0} for interval in Config.INTERVALS}
        
        for ticker in TICKERS:
            info = self.metadata.get_ticker_info(ticker)
            last_updates = info.get('last_updates', {})
            
            for interval in Config.INTERVALS:
                last_update_str = last_updates.get(interval)
                
                if last_update_str:
                    last_update = datetime.fromisoformat(last_update_str)
                    age = datetime.now() - last_update
                    threshold = Config.UPDATE_THRESHOLDS.get(interval, timedelta(days=1))
                    
                    if age <= threshold:
                        freshness[interval]['fresh'] += 1
                    else:
                        freshness[interval]['stale'] += 1
                else:
                    freshness[interval]['never'] += 1
        
        return freshness
    
    def _get_problem_tickers(self) -> List[Dict]:
        problems = []
        
        for ticker, info in self.metadata.data.items():
            error_count = info.get('consecutive_errors', 0)
            if error_count >= 3:
                problems.append({
                    'ticker': ticker,
                    'error_count': error_count,
                    'blacklisted': bool(info.get('blacklisted_until'))
                })
        
        return sorted(problems, key=lambda x: x['error_count'], reverse=True)
    
    def _generate_interval_status(self, freshness_data: Dict) -> str:
        html = ""
        
        for interval, data in freshness_data.items():
            total = data['fresh'] + data['stale'] + data['never']
            fresh_pct = (data['fresh'] / total * 100) if total > 0 else 0
            
            html += f"""
            <div class="status-item">
                <strong>{interval}</strong><br>
                Fresh: {data['fresh']} ({fresh_pct:.1f}%)<br>
                Stale: {data['stale']}<br>
                Never: {data['never']}
            </div>
            """
        
        return html
    
    def _generate_problems_section(self, problems: List[Dict]) -> str:
        if not problems:
            return ""
        
        html = """
        <div class="section">
            <h2>Problem Tickers</h2>
            <div class="problems">
        """
        
        for problem in problems[:15]:
            status = "Blacklisted" if problem['blacklisted'] else "Errors"
            html += f"""
            <div class="problem-item">
                <strong>{problem['ticker']}</strong> - {status} ({problem['error_count']} errors)
            </div>
            """
        
        html += """
            </div>
        </div>
        """
        
        return html


class MarketHoursBlocker:
    """Bloque les téléchargements inutiles pendant les fermetures de marché"""
    
    def __init__(self):
        self.logger = logging.getLogger("MarketHoursBlocker")
    
    def should_skip_ticker(self, ticker: str, interval: str, metadata_manager) -> bool:
        """
        MÉTHODE MANQUANTE - implémentation complète
        Détermine si un ticker doit être sauté selon les horaires de marché
        """
        now = datetime.now()
        
        # Récupérer la dernière mise à jour
        info = metadata_manager.get_ticker_info(ticker)
        last_update_str = info['last_updates'].get(interval)
        
        if not last_update_str:
            return False  # Pas de données = télécharger
        
        last_update = datetime.fromisoformat(last_update_str)
        
        # === LOGIQUE WEEKEND ===
        if self._is_weekend(now):
            return self._should_skip_weekend(last_update, now)
        
        # === LOGIQUE SEMAINE (heures de fermeture) ===
        elif self._is_market_closed_hours(now):
            return self._should_skip_night_hours(last_update, now)
        
        return False  # Marché ouvert = pas de skip
    
    def _is_weekend(self, dt: datetime) -> bool:
        """Vérifie si c'est le weekend (samedi/dimanche)"""
        return dt.weekday() >= 5  # 5=samedi, 6=dimanche
    
    def _is_market_closed_hours(self, dt: datetime) -> bool:
        """Vérifie si on est dans les heures de fermeture (22h-6h)"""
        hour = dt.hour
        return hour >= 22 or hour < 6
    
    def _should_skip_weekend(self, last_update: datetime, now: datetime) -> bool:
        """Logique weekend : skip si mis à jour depuis vendredi 22h"""
        # Trouver le vendredi 22h de cette semaine
        days_since_monday = now.weekday()
        if days_since_monday >= 5:  # Weekend
            friday = now - timedelta(days=days_since_monday - 4)  # Vendredi
        else:
            # Si on est en semaine, prendre vendredi dernier
            friday = now - timedelta(days=days_since_monday + 3)
        
        friday_22h = friday.replace(hour=22, minute=0, second=0, microsecond=0)
        
        # Skip si dernière MAJ >= vendredi 22h
        skip = last_update >= friday_22h
        
        if skip:
            self.logger.debug(f"Weekend skip: last update {last_update.strftime('%a %H:%M')} >= Fri 22:00")
        
        return skip
    
    def _should_skip_night_hours(self, last_update: datetime, now: datetime) -> bool:
        """Logique heures de nuit : skip si mis à jour dans les dernières 8h"""
        hours_since_update = (now - last_update).total_seconds() / 3600
        
        # Skip si mis à jour dans les 8 dernières heures pendant fermeture
        skip = hours_since_update < 8
        
        if skip:
            self.logger.debug(f"Night hours skip: last update {hours_since_update:.1f}h ago")
        
        return skip
    
    def get_market_status(self) -> str:
        """Retourne le statut du marché pour info"""
        now = datetime.now()
        
        if self._is_weekend(now):
            return "WEEKEND_CLOSED"
        elif self._is_market_closed_hours(now):
            return "NIGHT_CLOSED"
        else:
            return "MARKET_OPEN"
        
class OptimizedDownloader:
    def __init__(self):
        self.cache = SmartCache(max_size=1000, ttl_hours=24)
        self.metadata = MetadataManager()
        self.file_manager = FileManager(self.cache)
        self.technical_calculator = TechnicalIndicatorCalculator()
        self.metadata_enricher = MetadataEnricher(self.metadata)
        self.dashboard_generator = DashboardGenerator(self.metadata)
        self.logger = logging.getLogger("OptimizedDownloader")
        self.market_blocker = MarketHoursBlocker()

        # ✅ NOUVEAU: Initialiser l'enrichissement avancé
        self.earnings_cache = EarningsCalendarCache()
        # ✅ MODIFIER: Utiliser enrichissement avancé
        self.metadata_enricher = MetadataEnricher(self.metadata)
        
        # Créer les répertoires nécessaires
        for directory in [Config.DATA_DIR, Config.TECH_DATA_DIR]:
            os.makedirs(directory, exist_ok=True)
            for interval in Config.INTERVALS:
                os.makedirs(os.path.join(directory, interval), exist_ok=True)
        
        self.stats = {
            'downloads_attempted': 0,
            'downloads_successful': 0,
            'cache_hits': 0,
            'api_calls_saved': 0,
            'technical_calculated': 0,
            'metadata_enriched': 0
        }

    def _enrich_existing_files_periodically(self):
        """Enrichissement périodique des fichiers existants non enrichis ou obsolètes"""
        candidates_for_enrichment = []
        
        # Identifier les tickers nécessitant un enrichissement
        for ticker in TICKERS:
            if self.metadata.is_blacklisted(ticker):
                continue
                
            # Vérifier si le ticker a des données dans au moins un intervalle
            has_data = any(
                os.path.exists(os.path.join(Config.DATA_DIR, interval, f"{ticker}.parquet"))
                for interval in Config.INTERVALS
            )
            
            if has_data and (
                self.metadata.should_update_sector(ticker) or 
                self.metadata.should_update_earnings(ticker)
            ):
                candidates_for_enrichment.append(ticker)
        
        if candidates_for_enrichment:
            # Limiter à 100 tickers par session pour éviter la surcharge API
            limited_candidates = candidates_for_enrichment[:100]
            self.logger.info(f"Periodic enrichment for {len(limited_candidates)} tickers with existing data")
            
            self.metadata_enricher.update_metadata_from_api(limited_candidates)
            self.metadata_enricher.propagate_sectors_to_all_intervals()
        else:
            self.logger.info("No tickers require periodic enrichment")

    def run_complete_pipeline(self):
        """Pipeline optimisé avec priorisation en première étape"""
        self.logger.info("Starting optimized pipeline with priority-first approach")
        
        # Phase 1: PRIORISATION GLOBALE (nouveau - première étape critique)
        self.logger.info("Phase 1: Global prioritization across all intervals")
        global_priority_queues = self._build_global_priority_queues()
        
        if not any(global_priority_queues.values()):
            self.logger.info("No updates needed across all intervals")
            return self._generate_final_results(0, "")
        
        # Phase 2: Téléchargements optimisés selon priorités
        self.logger.info("Phase 2: Priority-based downloads")
        total_downloads = self._execute_priority_downloads(global_priority_queues)
        
        # Phase 3: Propagation secteurs (après téléchargements)
        self.logger.info("Phase 3: Propagating sectors post-download")
        self.metadata_enricher.propagate_sectors_to_all_intervals()
        
        # Phase 4: Enrichissement complet (nouveaux + périodique)
        self.logger.info("Phase 4: Comprehensive enrichment (new + periodic)")
        self._enrich_new_data_only()
        self._enrich_existing_files_periodically()
        
        # Phase 5: Traitement technique différé
        self.logger.info("Phase 5: Deferred technical calculations")
        self._calculate_technical_indicators()
        
        # Phase 6: Finalisation
        self.logger.info("Phase 6: Final reporting and cleanup")
        dashboard_path = self._finalize_pipeline()
        
        return self._generate_final_results(total_downloads, dashboard_path)
    
    def _build_global_priority_queues(self) -> Dict[str, List[str]]:
        """Construit les queues de priorité globales avec source unique de vérité"""
        global_queues = {}
        
        for interval in Config.INTERVALS:
            candidates = []
            
            for ticker in TICKERS:
                # Source unique de vérité pour la fraîcheur
                priority_score = self._calculate_unified_priority_score(ticker, interval)
                
                if priority_score is not None:
                    candidates.append((ticker, priority_score))
            
            # Tri par priorité décroissante
            candidates.sort(key=lambda x: x[1], reverse=True)
            global_queues[interval] = [ticker for ticker, _ in candidates]
            
            if global_queues[interval]:
                self.logger.info(f"Prioritized {len(global_queues[interval])} tickers for {interval}")
        
        return global_queues
    
    def _download_batch_mode(self, interval: str, queue: List[str]) -> int:
        """Mode téléchargement par lots optimisé"""
        self.logger.info(f"Using batch mode for {interval}: {len(queue)} tickers")
        
        batch_size = Config.BATCH_SIZES[interval]
        downloads = 0
        
        with tqdm(total=len(queue), desc=f"Batch {interval}") as pbar:
            for i in range(0, len(queue), batch_size):
                batch = queue[i:i+batch_size]
                results = self._download_from_api(batch, interval)
                
                for ticker, df in results.items():
                    enriched_df = self.metadata_enricher.enrich_dataframe_advanced(df, ticker)
                    if self.file_manager.save_data(ticker, interval, enriched_df):
                        self.metadata.update_success(ticker, interval)
                        downloads += 1
                        self.stats['downloads_successful'] += 1
                        self.stats['metadata_enriched'] += 1
                
                # Marquer les échecs
                failed_tickers = set(batch) - set(results.keys())
                for ticker in failed_tickers:
                    self.metadata.update_failure(ticker)
                
                pbar.update(len(batch))
                
                if i + batch_size < len(queue):
                    time.sleep(1)
        
        return downloads
    
    def run_with_quotas(self, total_batches=10000):
        """Mode quota - allocation proportionnelle par intervalle"""
        self.logger.info(f"Running quota mode with {total_batches} total batches")
        
        priority_queues = self._build_global_priority_queues()
        
        # Calcul des quotas proportionnels
        total_tickers = sum(len(queue) for queue in priority_queues.values())
        if total_tickers == 0:
            self.logger.info("No updates needed")
            return 0
        
        downloads = 0
        for interval, queue in priority_queues.items():
            if not queue:
                continue
                
            quota = max(1, int((len(queue) / total_tickers) * total_batches))
            limited_queue = queue[:quota * Config.BATCH_SIZES[interval]]
            
            downloads += self._download_batch_mode(interval, limited_queue)
        
        return downloads

    def run_sequential_all_intervals(self, max_tickers_per_interval=None):
        """Mode séquentiel - un ticker après l'autre, tous intervalles"""
        priority_queues = self._build_global_priority_queues()
        
        downloads = 0
        for interval, queue in priority_queues.items():
            if max_tickers_per_interval:
                queue = queue[:max_tickers_per_interval]
            downloads += self._download_sequential_mode(interval, queue)
        
        return downloads

    def run_parallel_all_intervals(self, max_workers=5):
        """Mode parallèle - téléchargements simultanés par intervalle"""
        priority_queues = self._build_global_priority_queues()
        
        downloads = 0
        for interval, queue in priority_queues.items():
            downloads += self._download_parallel_mode(interval, queue, max_workers)
        
        return downloads

    def _download_parallel_mode(self, interval: str, queue: List[str], max_workers: int = 5) -> int:
        """Mode téléchargement parallèle optimisé"""
        self.logger.info(f"Using parallel mode for {interval}: {len(queue)} tickers")
        
        downloads = 0
        
        def download_single(ticker: str):
            try:
                time.sleep(random.uniform(0.1, 0.3))
                df = self._download_ticker_directly(ticker, interval)
                
                if df is not None and not df.empty:
                    enriched_df = self.metadata_enricher.enrich_dataframe_advanced(df, ticker)
                    if self.file_manager.save_data(ticker, interval, enriched_df):
                        self.metadata.update_success(ticker, interval)
                        return True
                else:
                    self.metadata.update_failure(ticker)
                    return False
            except Exception as e:
                self.logger.error(f"Error downloading {ticker}: {str(e)}")
                self.metadata.update_failure(ticker)
                return False
        
        with tqdm(total=len(queue), desc=f"Parallel {interval}") as pbar:
            for i in range(0, len(queue), max_workers * 2):
                batch = queue[i:i + max_workers * 2]
                
                with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
                    futures = {executor.submit(download_single, ticker): ticker for ticker in batch}
                    
                    for future in concurrent.futures.as_completed(futures):
                        try:
                            if future.result():
                                downloads += 1
                                self.stats['downloads_successful'] += 1
                                self.stats['metadata_enriched'] += 1
                        except:
                            pass
                        pbar.update(1)
                
                if i + max_workers * 2 < len(queue):
                    time.sleep(1)
        
        return downloads
    
    def _download_sequential_mode(self, interval: str, queue: List[str]) -> int:
        """Mode téléchargement séquentiel pour petites queues"""
        self.logger.info(f"Using sequential mode for {interval}: {len(queue)} tickers")
        
        downloads = 0
        
        for ticker in tqdm(queue, desc=f"Sequential {interval}"):
            try:
                df = self._download_ticker_directly(ticker, interval)
                
                if df is not None and not df.empty:
                    # ✅ NOUVEAU: Utiliser enrichissement avancé
                    enriched_df = self.metadata_enricher.enrich_dataframe_advanced(df, ticker)
                    if self.file_manager.save_data(ticker, interval, enriched_df):
                        self.metadata.update_success(ticker, interval)
                        downloads += 1
                        self.stats['downloads_successful'] += 1
                        self.stats['metadata_enriched'] += 1
                else:
                    # ✅ NOUVEAU: Utiliser classification erreurs
                    self.metadata.update_failure(ticker, "No data returned")
                
                time.sleep(0.5)
                
            except Exception as e:
                self.logger.error(f"Error downloading {ticker}: {str(e)}")
                self.metadata.update_failure(ticker)
        
        return downloads
    
    def _download_ticker_directly(self, ticker: str, interval: str) -> Optional[pd.DataFrame]:
        """Téléchargement simplifié - priorisation déjà effectuée en amont"""
        try:
            # Plus de vérifications de fraîcheur ici - déjà fait en priorisation
            existing_df = self.file_manager.load_data(ticker, interval)
            
            if existing_df is not None and not existing_df.empty:
                # Téléchargement incrémental seulement
                last_date = existing_df.index.max()
                start_date = last_date + self._get_interval_delta(interval)
                end_date = datetime.now()
                
                if start_date >= end_date:
                    # Cas edge - marquage pour éviter re-traitement
                    self.stats['api_calls_saved'] += 1
                    return None
                
                df = yf.download(
                    ticker, 
                    start=start_date.strftime('%Y-%m-%d'),
                    end=end_date.strftime('%Y-%m-%d'),
                    interval=interval, 
                    progress=False
                )
            else:
                # Premier téléchargement avec période optimisée
                period = PeriodOptimizer.get_safe_period(interval, ticker)
                df = yf.download(ticker, interval=interval, period=period, progress=False)
            
            return self._process_data(df)
            
        except Exception as e:
            # Transmission de l'erreur pour classification
            self.metadata.update_failure(ticker, str(e))
            return None

    def _get_interval_delta(self, interval: str) -> timedelta:
        """Retourne le delta temporel pour un intervalle"""
        deltas = {
            '1m': timedelta(minutes=1),
            '5m': timedelta(minutes=5), 
            '15m': timedelta(minutes=15),
            '30m': timedelta(minutes=30),
            '1h': timedelta(hours=1),
            '1d': timedelta(days=1)
        }
        return deltas.get(interval, timedelta(days=1))

    def _get_optimized_period(self, interval: str) -> str:
        """Périodes optimisées pour premier téléchargement"""
        periods = {
            '1m': '5d',     # Seulement 5 jours au lieu de 7
            '5m': '30d',    # 30 jours au lieu de 60  
            '15m': '30d',   # 3 mois raisonnable
            '30m': '30d',   # 6 mois
            '1h': '1y',     # 1 an au lieu de 730d
            '1d': '5y'      # 5 ans au lieu de 10y
        }
        return periods.get(interval, '1y')

    def _download_ticker_fallback(self, ticker: str, interval: str) -> Optional[pd.DataFrame]:
        """Fallback vers téléchargement complet"""
        try:
            period = self._get_optimized_period(interval)
            df = yf.download(ticker, interval=interval, period=period, progress=False)
            return self._process_data(df)
        except Exception as e:
            self.logger.error(f"Fallback download failed for {ticker}: {str(e)}")
            return None
    
    def _download_from_api(self, tickers: List[str], interval: str) -> Dict[str, pd.DataFrame]:
        """Téléchargement groupé depuis l'API"""
        results = {}
        
        try:
            period_map = {
                '1m': '7d', '5m': '60d', '15m': '60d',
                '30m': '60d', '1h': '730d', '1d': '10y'
            }
            period = period_map.get(interval, '1y')
            
            data = yf.download(
                tickers=" ".join(tickers),
                interval=interval,
                period=period,
                group_by='ticker',
                prepost=False,
                threads=True,
                timeout=15,
                progress=False
            )
            
            if len(tickers) == 1 and isinstance(data, pd.DataFrame):
                ticker = tickers[0]
                processed = self._process_data(data)
                if not processed.empty:
                    results[ticker] = processed
            else:
                for ticker in tickers:
                    if ticker in data:
                        processed = self._process_data(data[ticker])
                        if not processed.empty:
                            results[ticker] = processed
            
        except Exception as e:
            self.logger.error(f"Batch download error for {interval}: {str(e)}")
        
        return results
    
    def _process_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Traitement et nettoyage des données"""
        if df is None or df.empty:
            return pd.DataFrame()
        
        try:
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(0)
            
            required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            available_cols = [col for col in required_cols if col in df.columns]
            
            if len(available_cols) < 5:
                return pd.DataFrame()
            
            df = df[available_cols].dropna(how='all')
            
            if not isinstance(df.index, pd.DatetimeIndex):
                df.index = pd.to_datetime(df.index)
            
            return df.sort_index()
            
        except Exception as e:
            self.logger.warning(f"Error processing data: {str(e)}")
            return pd.DataFrame()
    
    def _calculate_unified_priority_score(self, ticker: str, interval: str) -> Optional[float]:
        """Source unique de vérité pour la priorité avec optimisation première exécution"""
        
        # Vérification blacklist en premier
        if self.metadata.is_blacklisted(ticker):
            return None
        
        # Vérification rapide de l'existence et fraîcheur du fichier
        file_path = os.path.join(Config.DATA_DIR, interval, f"{ticker}.parquet")
        
        if not os.path.exists(file_path):
            return float('inf')  # Fichier manquant = priorité maximale
        
        # Vérification de la fraîcheur basée sur la modification du fichier
        try:
            file_mtime = datetime.fromtimestamp(os.path.getmtime(file_path))
            last_market_close = self._get_last_market_close(datetime.now())
            
            # Si le fichier a été modifié après la dernière fermeture de marché, il est à jour
            if file_mtime >= last_market_close:
                return None  # Fichier à jour, pas de téléchargement nécessaire
            
            # Calculer l'âge depuis la dernière fermeture de marché
            hours_outdated = (last_market_close - file_mtime).total_seconds() / 3600
            
            # Seuil minimum avant mise à jour
            required_threshold = self._get_required_update_threshold(interval)
            
            if hours_outdated < required_threshold:
                return None  # Pas assez ancien pour justifier une mise à jour
            
            # Score de priorité basé sur l'ancienneté
            interval_bonus = {'1m': 100, '5m': 80, '15m': 60, '30m': 40, '1h': 20, '1d': 10}.get(interval, 0)
            error_penalty = self.metadata.get_ticker_info(ticker).get('consecutive_errors', 0) * 5
            
            return hours_outdated + interval_bonus - error_penalty
            
        except Exception as e:
            # Erreur de lecture du fichier = priorité élevée pour re-téléchargement
            return 500.0
    
    def _get_last_market_close(self, current_time: datetime) -> datetime:
        """Retourne la dernière fermeture de marché effective"""
        today = current_time.date()
        weekday = today.weekday()  # 0=lundi, 6=dimanche
        
        if weekday <= 4:  # Lundi à vendredi
            if current_time.hour >= 23:  # Après 23h = marché fermé aujourd'hui
                close_date = today
            else:  # Avant 23h = marché pas encore fermé, prendre hier
                if weekday == 0:  # Lundi matin
                    close_date = today - timedelta(days=3)  # Vendredi précédent
                else:
                    close_date = today - timedelta(days=1)  # Hier
        elif weekday == 5:  # Samedi
            close_date = today - timedelta(days=1)  # Vendredi
        else:  # Dimanche
            close_date = today - timedelta(days=2)  # Vendredi
        
        # Retourner vendredi 23h (fermeture effective)
        return datetime.combine(close_date, datetime.min.time()) + timedelta(hours=23)
    
    def _get_required_update_threshold(self, interval: str) -> float:
        """Seuils minimums en heures avant autorisation de mise à jour"""
        thresholds = {
            '1m': 0.5,    # 30 minutes
            '5m': 1.0,    # 1 heure  
            '15m': 2.0,   # 2 heures
            '30m': 4.0,   # 4 heures
            '1h': 8.0,    # 8 heures
            '1d': 20.0    # 20 heures
        }
        return thresholds.get(interval, 4.0)

    
    def _calculate_technical_indicators(self):
        """Calcule les indicateurs techniques pour tous les fichiers nécessaires"""
        for interval in Config.INTERVALS:
            source_folder = os.path.join(Config.DATA_DIR, interval)
            target_folder = os.path.join(Config.TECH_DATA_DIR, interval)
            
            if not os.path.exists(source_folder):
                continue
            
            files = [f for f in os.listdir(source_folder) if f.endswith('.parquet')]
            
            if not files:
                continue
            
            calculated_count = 0
            
            for filename in tqdm(files, desc=f"Technical {interval}"):
                ticker = filename[:-8]
                source_file = os.path.join(source_folder, filename)
                target_file = os.path.join(target_folder, filename)
                
                if self.technical_calculator.needs_calculation(source_file, target_file):
                    try:
                        df = pd.read_parquet(source_file)
                        
                        if len(df) >= 20:
                            enriched_df = self.technical_calculator.calculate_all_indicators(df)
                            self.file_manager.save_data(ticker, interval, enriched_df, is_technical=True)
                            calculated_count += 1
                            self.stats['technical_calculated'] += 1
                    
                    except Exception as e:
                        self.logger.warning(f"Error calculating technical indicators for {ticker} ({interval}): {str(e)}")
            
            if calculated_count > 0:
                self.logger.info(f"Calculated technical indicators for {calculated_count} files in {interval}")
    
    def _enrich_new_data_only(self):
        """Enrichissement uniquement sur les données nouvellement téléchargées - remplace _enrich_metadata_selective()"""
        # Identifier les tickers qui ont été effectivement mis à jour cette session
        recently_updated_tickers = []
        
        for ticker in TICKERS:
            # Vérifier si mis à jour dans les dernières 30 minutes
            info = self.metadata.get_ticker_info(ticker)
            for interval, last_update_str in info.get('last_updates', {}).items():
                if last_update_str:
                    last_update = datetime.fromisoformat(last_update_str)
                    if (datetime.now() - last_update).total_seconds() < 1800:  # 30 minutes
                        recently_updated_tickers.append(ticker)
                        break
        
        if recently_updated_tickers:
            # Limiter l'enrichissement aux tickers récemment mis à jour
            limited_tickers = list(set(recently_updated_tickers))[:30]  # Max 30 pour éviter surcharge API
            self.logger.info(f"Enriching metadata for {len(limited_tickers)} recently updated tickers")
            
            self.metadata_enricher.update_metadata_from_api(limited_tickers)
        else:
            self.logger.info("No recent updates detected - skipping metadata enrichment")

    def _execute_priority_downloads(self, priority_queues: Dict[str, List[str]]) -> int:
        """Exécute les téléchargements selon les priorités établies"""
        total_downloads = 0
        
        for interval, queue in priority_queues.items():
            if not queue:
                continue
                
            # Sélection du mode selon taille de queue
            if len(queue) > 100 and interval in ['1d', '1h']:
                total_downloads += self._download_batch_mode(interval, queue)
            elif len(queue) > 50:
                total_downloads += self._download_parallel_mode(interval, queue)
            else:
                total_downloads += self._download_sequential_mode(interval, queue)
        
        return total_downloads
    
    def _generate_final_results(self, downloads: int, dashboard_path: str) -> Dict:
        """Génère les résultats finaux du pipeline"""
        cache_stats = self.cache.get_stats()
        self.stats['cache_hit_rate'] = cache_stats['hit_rate']
        
        return {
            'downloads': downloads,
            'dashboard': dashboard_path,
            'stats': self.stats
        }

    def _finalize_pipeline(self) -> str:
        """Finalise le pipeline et génère le dashboard"""
        self.metadata.save()
        return self.dashboard_generator.generate_dashboard(self.stats)

    def _print_pipeline_summary(self, total_downloads: int, dashboard_path: str):
        """Affiche le résumé du pipeline"""
        cache_stats = self.cache.get_stats()
        
        print("\n" + "="*60)
        print("PIPELINE EXECUTION SUMMARY")
        print("="*60)
        print(f"Successful downloads: {self.stats['downloads_successful']}")
        print(f"API calls saved: {self.stats['api_calls_saved']}")
        print(f"Technical indicators calculated: {self.stats['technical_calculated']}")
        print(f"Files enriched: {self.stats['metadata_enriched']}")
        print(f"Cache hit rate: {cache_stats['hit_rate']:.1f}%")
        print(f"Dashboard generated: {dashboard_path}")
        print("="*60)

def main():
    """Fonction principale avec choix des modes"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    
    # === CONFIGURATION DES MODES ===
    use_auto_pipeline = True      # Pipeline automatique optimisé
    use_quotas = False           # Allocation proportionnelle
    use_sequential = False       # Séquentiel simple
    use_parallel = False         # Parallèle multi-threads
    
    # Paramètres
    max_workers = 11
    max_tickers_per_interval = None
    total_batches = 10000
    
    downloader = OptimizedDownloader()
    
    if use_auto_pipeline:
        # Mode automatique (défaut) - garde tout l'original
        results = downloader.run_complete_pipeline()
        
    else:
        # Modes manuels
        if use_quotas:
            downloads = downloader.run_with_quotas(total_batches)
        elif use_sequential:
            downloads = downloader.run_sequential_all_intervals(max_tickers_per_interval)
        elif use_parallel:
            downloads = downloader.run_parallel_all_intervals(max_workers)
        
        # Finalisation manuelle pour récupérer dashboard et stats
        dashboard_path = downloader._finalize_pipeline()
        cache_stats = downloader.cache.get_stats()
        downloader.stats['cache_hit_rate'] = cache_stats['hit_rate']
        
        results = {
            'downloads': downloads,
            'dashboard': dashboard_path,
            'stats': downloader.stats
        }
    
    # === AFFICHAGE ORIGINAL PRÉSERVÉ ===
    print(f"\nPipeline completed successfully:")
    print(f"- {results['downloads']} files updated")
    print(f"- Dashboard: {results['dashboard']}")
    print(f"- {results['stats']['api_calls_saved']} API calls saved")

if __name__ == "__main__":
    main()

2025-05-26 13:11:06,065 - OptimizedDownloader - INFO - Starting optimized pipeline with priority-first approach
2025-05-26 13:11:06,065 - OptimizedDownloader - INFO - Phase 1: Global prioritization across all intervals
2025-05-26 13:11:06,286 - OptimizedDownloader - INFO - Prioritized 40 tickers for 1m
2025-05-26 13:11:06,286 - OptimizedDownloader - INFO - Phase 2: Priority-based downloads
2025-05-26 13:11:06,286 - OptimizedDownloader - INFO - Using sequential mode for 1m: 40 tickers
Sequential 1m:  20%|██        | 8/40 [00:05<00:20,  1.59it/s]2025-05-26 13:11:11,518 - yfinance - ERROR - 
1 Failed download:
2025-05-26 13:11:11,518 - yfinance - ERROR - ['CIHHF']: YFPricesMissingError('possibly delisted; no price data found  (period=5d)')
2025-05-26 13:11:11,518 - MetadataManager - WARNING - Progressive blacklist for CIHHF: 13h (consecutive: 3, recent: 3, type: temporary)
Sequential 1m:  85%|████████▌ | 34/40 [00:21<00:03,  1.54it/s]2025-05-26 13:11:28,297 - yfinance - ERROR - 
1 Failed 